In [1]:
!wget https://raw.githubusercontent.com/Mariogarber/PLN-Project/main/requirements.txt -O requirements.txt
!pip install -r requirements.txt

--2025-11-29 10:09:49--  https://raw.githubusercontent.com/Mariogarber/PLN-Project/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     147  --.-KB/s    in 0s      

2025-11-29 10:09:49 (4.37 MB/s) - ‘requirements.txt’ saved [147/147]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6

In [2]:
from torch.utils.data import Dataset
import pandas as pd
from typing import List
import pandas as pd
from bert_score import score as bertscore
import sacrebleu
from rouge_score import rouge_scorer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import logging

TOKENIZER = T5Tokenizer.from_pretrained("google/mt5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("google/mt5-base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
class ToxicNonToxicDataset(Dataset):
    """
    Dataset para tareas de detoxificación de texto.
    Espera varios archivos .parquet con columnas:
        - 'toxic_sentences': texto tóxico original (string)
        - 'neutral_sentences': versión detoxificada del texto (string)
    Cada fila es un par de oraciones (tóxica -> neutral).
    """

    def __init__(self, parquet_paths: List[str]):
        # Cargar y mezclar todos los parquets
        dfs = []
        for p in parquet_paths:
            df = pd.read_parquet(p)
            lang = p.split('_')[-1].split('.parquet')[0]
            df['language'] = lang
            dfs.append(df)
        self.data = pd.concat(dfs, ignore_index=True)
        
        # Limpiar datos: remover filas con valores nulos
        self.data = self.data.dropna(subset=["toxic_sentence", "neutral_sentence"])
        self.data = self.data.reset_index(drop=True)

        # Limpiar espacios en blanco
        self.data["toxic_sentence"] = self.data["toxic_sentence"].str.strip()
        self.data["neutral_sentence"] = self.data["neutral_sentence"].str.strip()

        # Mezclar datos
        self.data = self.data.sample(frac=1).reset_index(drop=True)

        # Asegurar tipos correctos
        self.data["toxic_sentence"] = self.data["toxic_sentence"].astype(str)
        self.data["neutral_sentence"] = self.data["neutral_sentence"].astype(str)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "toxic_sentence": self.data.iloc[idx]["toxic_sentence"],
            "neutral_sentence": self.data.iloc[idx]["neutral_sentence"],
            "language": self.data.iloc[idx]["language"]
        }
    
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)

In [4]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

def load_data():
    """Load multilingual detoxification dataset directly as HuggingFace Dataset with train/test split"""
    lang = ['en', 'am', 'ar', 'de', 'es', 'hi', 'ru', 'uk', 'zh']
    
    url_template = "https://raw.githubusercontent.com/Mariogarber/PLN-Project/main/dataset/toxic_nontoxic/multilingual_paradetox_{}.parquet"
    
    datasets = []
    for lang_code in lang:
        url = url_template.format(lang_code)
        try:
            # Load directly as HuggingFace dataset
            ds = load_dataset('parquet', data_files=url, split='train')
            # Add language column
            ds = ds.add_column("language", [lang_code] * len(ds))
            datasets.append(ds)
            LOGGER.info(f"Loaded {len(ds)} samples for language: {lang_code}")
        except Exception as e:
            LOGGER.warning(f"Failed to load data for {lang_code}: {e}")
    
    # Concatenate all language datasets
    combined_dataset = concatenate_datasets(datasets)
    
    # Shuffle the combined dataset
    combined_dataset = combined_dataset.shuffle(seed=42)
    
    # Split into train/test (80/20 split)
    train_test_split = combined_dataset.train_test_split(test_size=0.2, seed=42)
    
    # Create DatasetDict with train and test splits
    dataset_dict = DatasetDict({
        'train': train_test_split['train'],
        'test': train_test_split['test']
    })
    
    LOGGER.info(f"Total dataset size: {len(combined_dataset)} samples across {len(datasets)} languages")
    LOGGER.info(f"Train split: {len(dataset_dict['train'])} samples")
    LOGGER.info(f"Test split: {len(dataset_dict['test'])} samples")
    
    return dataset_dict

def load_forbidden_words():
    
    lang = ['en', 'am', 'ar', 'de', 'es', 'hi', 'ru', 'uk', 'zh']
    
    url_template = "https://raw.githubusercontent.com/Mariogarber/PLN-Project/main/dataset/multilingual_toxic_spans/{}.parquet"
    
    datasets = []
    for lang_code in lang:
        url = url_template.format(lang_code)
        try:
            # Load directly as HuggingFace dataset
            ds = load_dataset('parquet', data_files=url, split='train')
            # Add language column
            ds = ds.add_column("language", [lang_code] * len(ds))
            datasets.append(ds)
            LOGGER.info(f"Loaded {len(ds)} samples for language: {lang_code}")
        except Exception as e:
            LOGGER.warning(f"Failed to load data for {lang_code}: {e}")

    # Concatenate all language datasets
    combined_dataset = concatenate_datasets(datasets)
    LOGGER.info(f"Total forbidden words dataset size: {len(combined_dataset)} samples across {len(datasets)} languages")
    return combined_dataset

In [5]:
toxic_spans_dataset = load_forbidden_words()

dataset = load_data()

Generating train split: 0 examples [00:00, ? examples/s]

INFO:__main__:Loaded 991 samples for language: en


Generating train split: 0 examples [00:00, ? examples/s]

INFO:__main__:Loaded 995 samples for language: am


Generating train split: 0 examples [00:00, ? examples/s]

...

INFO:__main__:Loaded 400 samples for language: ru


Generating train split: 0 examples [00:00, ? examples/s]

INFO:__main__:Loaded 400 samples for language: uk


Generating train split: 0 examples [00:00, ? examples/s]

INFO:__main__:Loaded 400 samples for language: zh
INFO:__main__:Total dataset size: 3600 samples across 9 languages
INFO:__main__:Train split: 2880 samples
INFO:__main__:Test split: 720 samples


In [6]:
forbidden_words = toxic_spans_dataset['Negative Connotations']

# Convert to a flat list of unique words
forbidden_words_list = []
for words in forbidden_words:
    if words:  # Check if not None or empty
        # Split by common separators and add to list
        if isinstance(words, str):
            # Split by comma, semicolon, or other separators
            word_parts = words.replace(';', ',').replace('|', ',').split(',')
            for word in word_parts:
                word = word.strip().lower()
                if word and word not in forbidden_words_list:
                    forbidden_words_list.append(word)

forbidden_words_list = list(set(forbidden_words_list))

In [27]:
# 🚫 CUSTOM LOSS FUNCTION WITH FORBIDDEN WORDS PENALTY

import torch
import torch.nn.functional as F
from transformers import Trainer
import re
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

class ForbiddenLossTrainer(Trainer):
    """
    Custom Trainer with forbidden words penalty in loss function
    """
    
    def __init__(self, forbidden_tokens=None, penalty_weight=1.0, **kwargs):
        super().__init__(**kwargs)
        
        # Default forbidden words list (you can customize this)
        if forbidden_tokens is None:
            self.forbidden_words = [
                "hate", "stupid", "idiot", "moron", "dumb", "loser", 
                "worthless", "pathetic", "disgusting", "terrible",
                "fuck", "shit", "damn", "bitch", "asshole",
                "retard",
            ]
        else:
            self.forbidden_words = forbidden_words
            
        self.penalty_weight = penalty_weight
        
        self.forbidden_tokens = forbidden_tokens
        
        print(f"🚫 Custom loss initialized with {len(self.forbidden_words)} forbidden words")
        print(f"⚖️ Penalty weight: {penalty_weight}")
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Custom loss that penalizes the model for assigning probability
        to forbidden tokens. The penalty IS differentiable.
        """

        # Forward pass
        outputs = model(**inputs)
        original_loss = outputs.loss
        logits = outputs.logits  # [batch, seq_len, vocab]

        # If not training, return original loss
        if not self.model.training:
            return (original_loss, outputs) if return_outputs else original_loss

        probs = torch.softmax(logits, dim=-1)  # [B, L, V]

        device = logits.device
        vocab_size = logits.size(-1)

        # Creamos una máscara [V] donde forbidden_tokens tienen 1
        forbidden_mask = torch.zeros(vocab_size, device=device)
        forbidden_mask[self.forbidden_tokens] = 1  # <-- aquí debe ser una lista de IDs

        # expandimos a [B, L, V] mediante broadcast
        forbidden_mask = forbidden_mask.unsqueeze(0).unsqueeze(0)

        # Penalizamos la probabilidad asignada a esos tokens
        forbidden_prob = (probs * forbidden_mask).sum(dim=-1)  # [B, L]

        # Ignorar posiciones donde label = -100 (padding en training)
        if "labels" in inputs:
            labels = inputs["labels"]
            valid_mask = (labels != -100).float()  # [B, L]
            forbidden_prob = forbidden_prob * valid_mask

        # Penalización media por batch
        penalty = forbidden_prob.mean()

        total_loss = (1 - self.penalty_weight) * original_loss + self.penalty_weight * penalty

        return (total_loss, outputs) if return_outputs else total_loss

    # def _calculate_forbidden_penalty(self, predictions, labels=None):
    #     """
    #     Calculate penalty based on forbidden words in generated text
    #     """
    #     batch_size = predictions.shape[0]
    #     total_penalty = 0.0
        
    #     for i in range(batch_size):
    #         # Decode the prediction to text
    #         pred_ids = predictions[i]
            
    #         # Remove padding tokens and special tokens for cleaner text
    #         # Filter out -100 (ignored tokens) and padding
    #         valid_pred_ids = pred_ids[pred_ids >= 0]  # Remove -100 labels
    #         try:                
    #             # Count tokens that are in forbidden tokens
    #             forbidden_count = sum(1 for token_id in valid_pred_ids if token_id.item() in self.forbidden_tokens)
                
    #             # Add penalty proportional to number of forbidden words found
    #             if forbidden_count > 0:
    #                 total_penalty += forbidden_count
                    
    #         except Exception as e:
    #             # If decoding fails, skip this sample
    #             continue
        
    #     # Convert to tensor on the same device as the model
    #     device = predictions.device
    #     penalty_tensor = torch.tensor(total_penalty, dtype=torch.float32, device=device)
        
    #     return penalty_tensor

# Example usage configuration
FORBIDDEN_WORDS = forbidden_words_list
PENALTY_WEIGHT = 2.0  # How much to penalize forbidden words (adjust as needed)

print("✅ Custom loss function with forbidden words penalty created!")
print(f"📝 Forbidden words list contains {len(FORBIDDEN_WORDS)} words")
print(f"🎯 Example forbidden words: {FORBIDDEN_WORDS[:10]}...")
print(f"⚖️ Penalty weight: {PENALTY_WEIGHT}")
print("\n🔄 To use the custom trainer:")
print("custom_trainer = CustomLossTrainer(")
print("    model=model_lora,")
print("    args=training_args,")
print("    train_dataset=tokenized_dataset['train'],")
print("    eval_dataset=tokenized_dataset['eval'],")
print("    data_collator=data_collator,")
print("    tokenizer=TOKENIZER,")
print("    forbidden_words=FORBIDDEN_WORDS,")
print("    penalty_weight=PENALTY_WEIGHT")
print(")")

✅ Custom loss function with forbidden words penalty created!
📝 Forbidden words list contains 8869 words
🎯 Example forbidden words: ['ዲዳ', 'قديم', 'बहन बेटियो', 'жестокая', 'spoiled', 'golpista', 'ማፈሪያ', 'ሰይጣኑ', 'سرطان', 'dünnschiss']...
⚖️ Penalty weight: 2.0

🔄 To use the custom trainer:
custom_trainer = CustomLossTrainer(
    model=model_lora,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    data_collator=data_collator,
    tokenizer=TOKENIZER,
    forbidden_words=FORBIDDEN_WORDS,
    penalty_weight=PENALTY_WEIGHT
)


In [ ]:
import torch
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Create QLoRA configuration with bfloat16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for stability
    bnb_4bit_use_double_quant=False,        # Disabled for stability
)

# Load and quantize the model
model_quantized = T5ForConditionalGeneration.from_pretrained(
    "google/mt5-base",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# Prepare model for k-bit training
model_quantized = prepare_model_for_kbit_training(model_quantized)

# LoRA configuration
lora_config = LoraConfig(
    r=8,                    # Rank
    lora_alpha=16,          # Alpha parameter
    target_modules=["q", "v", "k", "o"],  # Target attention modules
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Apply LoRA to the quantized model
model_lora = get_peft_model(model_quantized, lora_config)

print("✅ QLoRA model created with bfloat16 configuration")
print(f"📊 Model type: {type(model_lora)}")
print(f"🔧 Compute dtype: {bnb_config.bnb_4bit_compute_dtype}")

# Check trainable parameters
trainable_params = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_lora.parameters())
print(f"🎯 Trainable parameters: {trainable_params:,} / {total_params:,} ({trainable_params/total_params*100:.2f}%)")

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


✅ QLoRA model created with bfloat16 configuration
📊 Model type: <class 'peft.peft_model.PeftModelForSeq2SeqLM'>
🔧 Compute dtype: torch.bfloat16
🎯 Trainable parameters: 884,736 / 503,069,952 (0.18%)


In [29]:
# Custom callback to debug training issues
from transformers import TrainerCallback, TrainingArguments, DataCollatorForSeq2Seq
import numpy as np
from transformers import Trainer

class DebugCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 10 == 0:  # Log every 10 steps
            logs = kwargs.get('logs', {})
            loss = logs.get('train_loss', 'N/A')
            
            print(f"Step {state.global_step}: Loss = {loss}")
            
            # Check for problematic loss values
            if isinstance(loss, (int, float)):
                if loss == 0:
                    print("⚠️  WARNING: Loss is exactly 0 - check your data preprocessing!")
                elif loss < 1e-6:
                    print("⚠️  WARNING: Loss is extremely small - possible numerical issues!")
                elif np.isnan(loss):
                    print("⚠️  ERROR: Loss is NaN - stopping training recommended!")
                    control.should_training_stop = True


# First, create the tokenized dataset
def tokenize_function(examples):
    """Tokenize the dataset for T5 training"""
    # Create input text with task prefix
    input_texts = [f"detoxify: {text}" for text in examples['toxic_sentence']]
    target_texts = examples['neutral_sentence']
    
    # Tokenize inputs
    model_inputs = TOKENIZER(
        input_texts,
        max_length=512,
        truncation=True,
        padding=False  # We'll pad in the data collator
    )
    
    # Tokenize targets
    with TOKENIZER.as_target_tokenizer():
        labels = TOKENIZER(
            target_texts,
            max_length=512,
            truncation=True,
            padding=False
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to the train dataset
print("🔄 Tokenizing dataset...")
tokenized_dataset = dataset['train'].map(
    tokenize_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

print(f"✅ Tokenized dataset created with {len(tokenized_dataset)} samples")

forbidden_word_tokens = []
for word in forbidden_words_list:
    token_ids = TOKENIZER.encode(word, add_special_tokens=False)
    forbidden_word_tokens.extend(token_ids)

# Split train into train and eval
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_dataset = DatasetDict({
    'train': train_test_split['train'],
    'eval': train_test_split['test']
})

# Add the debug callback to trainer
debug_callback = DebugCallback()

# Define training arguments
training_args = TrainingArguments(
    output_dir="./mt5-detoxify-qlora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=10,
    learning_rate=1e-4,
    bf16=True,
    logging_steps=200,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=200,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
)
# Create data collator
data_collator = DataCollatorForSeq2Seq(
    TOKENIZER, 
    model=model_lora,
    padding=True,
    return_tensors="pt"
)

🔄 Tokenizing dataset...
✅ Tokenized dataset created with 2880 samples


In [66]:
# 🎯 CREATE CUSTOM TRAINER WITH FORBIDDEN WORDS PENALTY

# First, let's create a custom trainer using your existing model
print("🔧 Creating custom trainer with forbidden words penalty...")

# Create custom trainer with forbidden words penalty
custom_trainer = ForbiddenLossTrainer(
    model=model_lora,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    data_collator=data_collator,
    tokenizer=TOKENIZER,
    forbidden_tokens=forbidden_word_tokens,
    penalty_weight=0.15,
    callbacks=[debug_callback]
)

print("✅ Custom trainer created successfully!")
print(f"🚫 Monitoring {len(FORBIDDEN_WORDS)} forbidden words")
print(f"⚖️ Penalty weight: {PENALTY_WEIGHT}")
print("\n🚀 Ready to train with custom loss! Run: custom_trainer.train()")

🔧 Creating custom trainer with forbidden words penalty...
🚫 Custom loss initialized with 8788 forbidden words
⚖️ Penalty weight: 0.15
✅ Custom trainer created successfully!
🚫 Monitoring 8869 forbidden words
⚖️ Penalty weight: 2.0

🚀 Ready to train with custom loss! Run: custom_trainer.train()


/tmp/ipython-input-891937490.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `ForbiddenLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(**kwargs)


In [ ]:
# 🔧 FIX VERIFIED: Test Custom Trainer Compatibility

print("🧪 Testing custom trainer compatibility with current Transformers version...")

try:
    # Test creating the custom trainer 
    test_trainer = ForbiddenLossTrainer(
        model=model_lora,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['eval'],
        data_collator=data_collator,
        tokenizer=TOKENIZER,
        forbidden_tokens=forbidden_word_tokens,
        penalty_weight=0.15,
        callbacks=[debug_callback]
    )
    
    # Test compute_loss method with a small batch
    test_samples = [tokenized_dataset['train'][i] for i in range(1)]
    test_batch = data_collator(test_samples)
    
    # Move to model device
    model_device = next(model_lora.parameters()).device
    test_batch = {k: v.to(model_device) if isinstance(v, torch.Tensor) else v 
                  for k, v in test_batch.items()}
    
    # Test the compute_loss method (this should now work without the TypeError)
    with torch.no_grad():
        loss = test_trainer.compute_loss(model_lora, test_batch)
        print(f"✅ Custom loss computation successful! Loss: {loss.item():.4f}")
    
    print("✅ All compatibility tests passed!")
    print("🚀 Custom trainer is ready for training!")
    
except Exception as e:
    print(f"❌ Error during testing: {e}")
    print("🔧 Please check the error and re-run the custom loss function cells")

🧪 Testing custom trainer compatibility with current Transformers version...
🚫 Custom loss initialized with 8788 forbidden words
⚖️ Penalty weight: 0.15
✅ Custom loss computation successful! Loss: 5.8383
✅ All compatibility tests passed!
🚀 Custom trainer is ready for training!


/tmp/ipython-input-3269202755.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(**kwargs)


In [67]:
custom_trainer.train()

Step,Training Loss,Validation Loss
200,10.682200,5.474257
400,5.443400,2.623299
600,2.996700,2.209522
800,2.551800,2.086040
1000,2.420500,1.996282
1200,2.298300,1.917997
1400,2.248300,1.905115


Step 10: Loss = N/A
Step 20: Loss = N/A
Step 30: Loss = N/A
Step 40: Loss = N/A
Step 50: Loss = N/A
Step 60: Loss = N/A
Step 70: Loss = N/A
Step 80: Loss = N/A
Step 90: Loss = N/A
Step 100: Loss = N/A
Step 110: Loss = N/A
Step 120: Loss = N/A
Step 130: Loss = N/A
Step 140: Loss = N/A
Step 150: Loss = N/A
Step 160: Loss = N/A
Step 170: Loss = N/A
Step 180: Loss = N/A
Step 190: Loss = N/A
Step 200: Loss = N/A
Step 210: Loss = N/A
Step 220: Loss = N/A
Step 230: Loss = N/A
Step 240: Loss = N/A
Step 250: Loss = N/A
Step 260: Loss = N/A
Step 270: Loss = N/A
Step 280: Loss = N/A
Step 290: Loss = N/A
Step 300: Loss = N/A
Step 310: Loss = N/A
Step 320: Loss = N/A
Step 330: Loss = N/A
Step 340: Loss = N/A
Step 350: Loss = N/A
Step 360: Loss = N/A
Step 370: Loss = N/A
Step 380: Loss = N/A
Step 390: Loss = N/A
Step 400: Loss = N/A
Step 410: Loss = N/A
Step 420: Loss = N/A
Step 430: Loss = N/A
Step 440: Loss = N/A
Step 450: Loss = N/A
Step 460: Loss = N/A
Step 470: Loss = N/A
Step 480: Loss = N/A
S

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Step 510: Loss = N/A
Step 520: Loss = N/A
Step 530: Loss = N/A
Step 540: Loss = N/A
Step 550: Loss = N/A
Step 560: Loss = N/A
Step 570: Loss = N/A
Step 580: Loss = N/A
Step 590: Loss = N/A
Step 600: Loss = N/A
Step 610: Loss = N/A
Step 620: Loss = N/A
Step 630: Loss = N/A
Step 640: Loss = N/A
Step 650: Loss = N/A
Step 660: Loss = N/A
Step 670: Loss = N/A
Step 680: Loss = N/A
Step 690: Loss = N/A
Step 700: Loss = N/A
Step 710: Loss = N/A
Step 720: Loss = N/A
Step 730: Loss = N/A
Step 740: Loss = N/A
Step 750: Loss = N/A
Step 760: Loss = N/A
Step 770: Loss = N/A
Step 780: Loss = N/A
Step 790: Loss = N/A
Step 800: Loss = N/A
Step 810: Loss = N/A
Step 820: Loss = N/A
Step 830: Loss = N/A
Step 840: Loss = N/A
Step 850: Loss = N/A
Step 860: Loss = N/A
Step 870: Loss = N/A
Step 880: Loss = N/A
Step 890: Loss = N/A
Step 900: Loss = N/A
Step 910: Loss = N/A
Step 920: Loss = N/A
Step 930: Loss = N/A
Step 940: Loss = N/A
Step 950: Loss = N/A
Step 960: Loss = N/A
Step 970: Loss = N/A
Step 980: Los

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Step 1010: Loss = N/A
Step 1020: Loss = N/A
Step 1030: Loss = N/A
Step 1040: Loss = N/A
Step 1050: Loss = N/A
Step 1060: Loss = N/A
Step 1070: Loss = N/A
Step 1080: Loss = N/A
Step 1090: Loss = N/A
Step 1100: Loss = N/A
Step 1110: Loss = N/A
Step 1120: Loss = N/A
Step 1130: Loss = N/A
Step 1140: Loss = N/A
Step 1150: Loss = N/A
Step 1160: Loss = N/A
Step 1170: Loss = N/A
Step 1180: Loss = N/A
Step 1190: Loss = N/A
Step 1200: Loss = N/A
Step 1210: Loss = N/A
Step 1220: Loss = N/A
Step 1230: Loss = N/A
Step 1240: Loss = N/A
Step 1250: Loss = N/A
Step 1260: Loss = N/A
Step 1270: Loss = N/A
Step 1280: Loss = N/A
Step 1290: Loss = N/A
Step 1300: Loss = N/A
Step 1310: Loss = N/A
Step 1320: Loss = N/A
Step 1330: Loss = N/A
Step 1340: Loss = N/A
Step 1350: Loss = N/A
Step 1360: Loss = N/A
Step 1370: Loss = N/A
Step 1380: Loss = N/A
Step 1390: Loss = N/A
Step 1400: Loss = N/A
Step 1410: Loss = N/A
Step 1420: Loss = N/A
Step 1430: Loss = N/A
Step 1440: Loss = N/A


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


TrainOutput(global_step=1440, training_loss=4.041978756586711, metrics={'train_runtime': 4660.8229, 'train_samples_per_second': 4.943, 'train_steps_per_second': 0.309, 'total_flos': 2049937178775552.0, 'train_loss': 4.041978756586711, 'epoch': 10.0})

In [ ]:
# apply the inference in the test dataset
# 🔮 INFERENCE ON TEST DATASET

print("🔮 Starting inference on test dataset...")

# Set model to evaluation mode
model_lora.eval()

# Get test dataset
test_dataset = dataset['test']
train_dataset = dataset['train']
# Filter test dataset to only include Spanish samples
test_dataset_es = test_dataset.filter(lambda x: x['language'] == 'es')
train_dataset_es = train_dataset.filter(lambda x: x['language'] == 'es')
print(f"📊 Spanish test dataset size: {len(test_dataset_es)} samples")
print(f"📊 Original test dataset size: {len(test_dataset)} samples")

# Update test_dataset to use the filtered version
test_dataset = test_dataset_es

bad_words = [[TOKENIZER.convert_tokens_to_ids(f"extra_id_{i}")] for i in range(100)]

# Function to generate detoxified text
def detoxify_text(toxic_text, max_length=512):
    """Generate detoxified version of toxic text"""
    # Add task prefix
    input_text = f"{toxic_text}"
    
    # Tokenize input
    inputs = TOKENIZER(
        input_text,
        return_tensors="pt",
        max_length=max_length,
        truncation=True,
        padding=True
    ).to(model_device)
    
    # Generate output
    with torch.no_grad():
        outputs = model_lora.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_beams=4,
            do_sample=False,
            early_stopping=True,
            pad_token_id=TOKENIZER.pad_token_id,
            eos_token_id=TOKENIZER.eos_token_id,
            bad_words_ids=bad_words
        )
    
    # Decode output
    generated_text = TOKENIZER.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Run inference on test samples
results = []
num_samples = max(10, len(test_dataset))

print(f"\n🧪 Testing on {num_samples} samples:")
print("=" * 80)

for i in range(num_samples):
    sample = train_dataset_es[i]
    toxic_sentence = sample['toxic_sentence']
    expected_neutral = sample['neutral_sentence']
    language = sample['language']
    
    # Generate detoxified version
    try:
        generated_neutral = detoxify_text(toxic_sentence)
        
        # Store results
        result = {
            'index': i,
            'language': language,
            'toxic_input': toxic_sentence,
            'expected_output': expected_neutral,
            'generated_output': generated_neutral
        }
        results.append(result)
        
        # Print results
        print(f"\nSample {i+1} ({language}):")
        print(f"🔴 Toxic:     {toxic_sentence[:100]}{'...' if len(toxic_sentence) > 100 else ''}")
        print(f"🟢 Expected:  {expected_neutral[:100]}{'...' if len(expected_neutral) > 100 else ''}")
        print(f"🔵 Generated: {generated_neutral[:100]}{'...' if len(generated_neutral) > 100 else ''}")
        
    except Exception as e:
        print(f"❌ Error processing sample {i}: {e}")
        continue

print("=" * 80)
print(f"✅ Inference completed on {len(results)} samples")


In [72]:
import numpy as np
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')

class DetoxificationEvaluator:
    def __init__(self, bert_model='all-MiniLM-L6-v2'):
        """
        Initialize the evaluator with ROUGE, BLEU, and BERT similarity metrics.
        
        Args:
            bert_model: The sentence transformer model to use for BERT similarity
        """
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.bert_model = SentenceTransformer(bert_model)
        self.smoothing = SmoothingFunction()
    
    def compute_rouge(self, reference, prediction):
        """Compute ROUGE scores (ROUGE-1, ROUGE-2, ROUGE-L)"""
        scores = self.rouge_scorer.score(reference, prediction)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }
    
    def compute_bleu(self, reference, prediction):
        """Compute BLEU score with smoothing"""
        ref_tokens = reference.split()
        pred_tokens = prediction.split()
        
        # Use smoothing to handle cases with no n-gram overlap
        bleu = sentence_bleu(
            [ref_tokens], 
            pred_tokens, 
            smoothing_function=self.smoothing.method1
        )
        return bleu
    
    def compute_bert_similarity(self, text1, text2):
        """Compute BERT-based semantic similarity using sentence embeddings"""
        embeddings = self.bert_model.encode([text1, text2])
        similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
        return similarity
    
    def compute_input_output_similarity(self, input_text, output_text):
        """
        Compute similarity ratio between input and output.
        Uses both character-level and semantic similarity.
        """
        # Character-level similarity (like difflib)
        char_similarity = SequenceMatcher(None, input_text, output_text).ratio()
        
        # Semantic similarity using BERT
        semantic_similarity = self.compute_bert_similarity(input_text, output_text)
        
        return {
            'character_similarity': char_similarity,
            'semantic_similarity': semantic_similarity,
            'average_similarity': (char_similarity + semantic_similarity) / 2
        }
    
    def evaluate_single(self, input_text, reference_text, prediction_text):
        """
        Evaluate a single prediction against reference and input.
        
        Args:
            input_text: The original toxic input
            reference_text: The ground truth detoxified text
            prediction_text: The model's predicted detoxified text
        
        Returns:
            Dictionary with all evaluation metrics
        """
        rouge_scores = self.compute_rouge(reference_text, prediction_text)
        bleu_score = self.compute_bleu(reference_text, prediction_text)
        bert_similarity = self.compute_bert_similarity(reference_text, prediction_text)
        input_output_sim = self.compute_input_output_similarity(input_text, prediction_text)
        
        return {
            'rouge1': rouge_scores['rouge1'],
            'rouge2': rouge_scores['rouge2'],
            'rougeL': rouge_scores['rougeL'],
            'bleu': bleu_score,
            'bert_similarity': bert_similarity,
            'input_output_char_similarity': input_output_sim['character_similarity'],
            'input_output_semantic_similarity': input_output_sim['semantic_similarity'],
            'input_output_avg_similarity': input_output_sim['average_similarity']
        }
    
    def evaluate_batch(self, inputs, references, predictions):
        """
        Evaluate a batch of predictions.
        
        Args:
            inputs: List of original toxic inputs
            references: List of ground truth detoxified texts
            predictions: List of model predictions
        
        Returns:
            Dictionary with averaged metrics and individual scores
        """
        results = []
        
        for inp, ref, pred in zip(inputs, references, predictions):
            result = self.evaluate_single(inp, ref, pred)
            results.append(result)
        
        # Compute average metrics
        avg_metrics = {
            metric: np.mean([r[metric] for r in results])
            for metric in results[0].keys()
        }
        
        return {
            'average_metrics': avg_metrics,
            'individual_scores': results
        }
    
    def print_summary(self, evaluation_results):
        """Print a formatted summary of evaluation results"""
        metrics = evaluation_results['average_metrics']
        
        print("=" * 60)
        print("DETOXIFICATION EVALUATION SUMMARY")
        print("=" * 60)
        print("\nContent Preservation Metrics (vs Reference):")
        print(f"  ROUGE-1:        {metrics['rouge1']:.4f}")
        print(f"  ROUGE-2:        {metrics['rouge2']:.4f}")
        print(f"  ROUGE-L:        {metrics['rougeL']:.4f}")
        print(f"  BLEU:           {metrics['bleu']:.4f}")
        print(f"  BERT Similarity: {metrics['bert_similarity']:.4f}")
        
        print("\nInput-Output Similarity (Content Retention):")
        print(f"  Character-level: {metrics['input_output_char_similarity']:.4f}")
        print(f"  Semantic:        {metrics['input_output_semantic_similarity']:.4f}")
        print(f"  Average:         {metrics['input_output_avg_similarity']:.4f}")
        print("=" * 60)

evaluator = DetoxificationEvaluator()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [74]:
test_dataset = dataset['test']
results = pd.DataFrame()

for i in range(len(test_dataset)):
    sample = test_dataset[i]
    toxic_sentence = sample['toxic_sentence']
    expected_neutral = sample['neutral_sentence']
    language = sample['language']
    
    # Generate detoxified version
    generated_neutral = detoxify_text(toxic_sentence)
    
    # Store results
    result = {
        'index': i,
        'language': language,
        'toxic_input': toxic_sentence,
        'expected_output': expected_neutral,
        'generated_output': generated_neutral
    }
    
    result = evaluator.evaluate_single(
        input_text=toxic_sentence,
        reference_text=expected_neutral,
        prediction_text=generated_neutral
    )

    results = pd.concat([results, pd.DataFrame([result])], ignore_index=True)

In [ ]:
results

,rouge1,rouge2,rougeL,bleu,bert_similarity,input_output_char_similarity,input_output_semantic_similarity,input_output_avg_similarity
0,0.000000,0.000000,0.000000,0.025099,0.775198,0.820896,0.842569,0.831732
1,0.000000,0.000000,0.000000,0.022417,1.000000,0.745098,0.936967,0.841033
2,0.000000,0.000000,0.000000,0.018675,0.666273,0.515337,0.757817,0.636577
3,0.000000,0.000000,0.000000,0.863340,0.917817,0.785714,0.861797,0.823756
4,0.844444,0.837209,0.844444,0.712205,0.940899,0.836364,0.927626,0.881995
...,...,...,...,...,...,...,...,...
715,0.000000,0.000000,0.000000,0.305098,0.741868,0.646154,0.919711,0.782932
716,0.000000,0.000000,0.000000,0.279528,0.819495,0.577778,0.857838,0.717808
717,0.000000,0.000000,0.000000,0.042830,0.851192,0.750000,0.843166,0.796583
718,0.615385,0.545455,0.615385,0.508133,0.533931,0.555556,0.340088,0.447822


### Focus Aproach on Spanish and English and adding the task

In [ ]:
import gc
import traceback

# Clear any existing models to avoid conflicts
torch.cuda.empty_cache()

dataset = load_data()

# Filter dataset for Spanish and English only
selected_languages = ['es', 'en']

filtered_dataset_train = dataset['train'].filter(lambda example: example['language'] in selected_languages)
filtered_dataset_test = dataset['test'].filter(lambda example: example['language'] in selected_languages)

# Tokenize the filtered dataset
tokenized_filtered_dataset = filtered_dataset_train.map(
    tokenize_function,
    batched=True,
    remove_columns=filtered_dataset_train.column_names
)

filtered_train_test_split = tokenized_filtered_dataset.train_test_split(test_size=0.2, seed=42)

tokenized_filtered_dataset_for_training = DatasetDict({
    'train': filtered_train_test_split['train'],
    'eval': filtered_train_test_split['test']
})

# Create a new model with 8-bit quantization (avoiding conflicts with previous models)
model_es_en = T5ForConditionalGeneration.from_pretrained(
    "google/mt5-base",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# LoRA configuration for the new model
lora_config_new = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Apply LoRA to the new model
model_es_en_lora = get_peft_model(model_es_en, lora_config_new)

print("✅ New model created successfully!")
print(f"🎯 Trainable parameters: {sum(p.numel() for p in model_es_en_lora.parameters() if p.requires_grad):,}")

# Create new data collator for this model
data_collator_es_en = DataCollatorForSeq2Seq(
    TOKENIZER, 
    model=model_es_en_lora,
    padding=True,
    return_tensors="pt"
)

# Create the trainer
trainer_es_en = ForbiddenLossTrainer(
    model=model_es_en_lora,
    args=training_args,
    train_dataset=tokenized_filtered_dataset_for_training['train'],
    eval_dataset=tokenized_filtered_dataset_for_training['eval'],
    data_collator=data_collator_es_en,
    tokenizer=TOKENIZER,
    forbidden_tokens=forbidden_word_tokens,
    penalty_weight=0.15,
    callbacks=[debug_callback]
)

print("✅ Spanish-English trainer created successfully!")

def check_trainable_params(model):
    count = 0
    for n, p in model.named_parameters():
        if p.requires_grad:
            count += 1
            print("TRAINABLE:", n)
    print("Total trainable:", count)

check_trainable_params(model_es_en_lora)


In [ ]:
trainer_es_en.train()

In [24]:
results_spanish_english = []
# Run inference on test samples for spanish and english
num_samples_se = min(10, len(filtered_dataset['eval']))  # Test on first 10 samples

for i in range(num_samples_se):
    sample = filtered_dataset['eval'][i]
    toxic_sentence = sample['toxic_sentence']
    expected_neutral = sample['neutral_sentence']
    language = sample['language']
    try:
        generated_neutral = detoxify_text(toxic_sentence)
        
        # Store results
        result = {
            'index': i,
            'language': language,
            'toxic_input': toxic_sentence,
            'expected_output': expected_neutral,
            'generated_output': generated_neutral
        }
        results_spanish_english.append(result)
        
        # Print results
        print(f"\nSample {i+1} ({language}):")
        print(f"🔴 Toxic:     {toxic_sentence[:100]}{'...' if len(toxic_sentence) > 100 else ''}")
        print(f"🟢 Expected:  {expected_neutral[:100]}{'...' if len(expected_neutral) > 100 else ''}")
        print(f"🔵 Generated: {generated_neutral[:100]}{'...' if len(generated_neutral) > 100 else ''}")
        
    except Exception as e:
        print(f"❌ Error processing sample {i}: {e}")
        continue

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Caching is incompatible with gradient checkpointing in T5Block. Setting `past_key_values=None`.



Sample 1 (es):
🔴 Toxic:     Que analogía rebuscada y de mierda
🟢 Expected:  Qué analogía confusa
🔵 Generated: <extra_id_0>   <extra_id_10> <extra_id_23> cien <extra_id_55> <extra_id_39> <extra_id_55> <extra_id_...

Sample 2 (en):
🔴 Toxic:     the guy was fucking shot ! .
🟢 Expected:  The guy was shot
🔵 Generated: <extra_id_0>  <extra_id_39> <extra_id_20>    <extra_id_1>

Sample 3 (es):
🔴 Toxic:     Debería darte vergüenza ofender las creencias de otras persona, intolerante fascista.
🟢 Expected:  Debería darte vergüenza ofender las creencias de otras persona.
🔵 Generated: <extra_id_0>

Sample 4 (en):
🔴 Toxic:     shit looked like a music video with synced audio , and footage editing .
🟢 Expected:  looked like a music video with synced audio , and footage editing .
🔵 Generated: <extra_id_0>  <extra_id_30> <extra_id_27>? <extra_id_10>. <extra_id_20>  <extra_id_54>. <extra_id_39...

Sample 5 (es):
🔴 Toxic:     Ya decía yo que veía menos moros
🟢 Expected:  Ya me había dado cuenta que veía 

In [ ]:
# 🔬 ADVANCED CUSTOM LOSS OPTIONS & TESTING

# =============================================
# OPTION 1: DYNAMIC PENALTY SCALING
# =============================================
class AdaptiveLossTrainer(CustomLossTrainer):
    """
    Advanced trainer with adaptive penalty that increases during training
    """
    
    def __init__(self, penalty_start=0.5, penalty_end=3.0, **kwargs):
        super().__init__(**kwargs)
        self.penalty_start = penalty_start
        self.penalty_end = penalty_end
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Calculate current penalty weight based on training progress
        if hasattr(self.args, 'num_train_epochs') and hasattr(self.state, 'epoch'):
            progress = self.state.epoch / self.args.num_train_epochs
            current_penalty = self.penalty_start + (self.penalty_end - self.penalty_start) * progress
            
            # Temporarily update penalty weight
            original_penalty = self.penalty_weight
            self.penalty_weight = current_penalty
            
            # Calculate loss with dynamic penalty
            result = super().compute_loss(model, inputs, return_outputs, **kwargs)
            
            # Restore original penalty weight
            self.penalty_weight = original_penalty
            
            return result
        else:
            return super().compute_loss(model, inputs, return_outputs, **kwargs)

# =============================================
# OPTION 2: CONTEXT-AWARE PENALTY
# =============================================
class ContextAwareLossTrainer(CustomLossTrainer):
    """
    Trainer that applies different penalties based on context
    """
    
    def __init__(self, context_penalties=None, **kwargs):
        super().__init__(**kwargs)
        
        # Define different penalty weights for different contexts
        self.context_penalties = context_penalties or {
            'high_toxicity': 5.0,    # Higher penalty for very toxic inputs
            'medium_toxicity': 2.0,  # Medium penalty for moderately toxic inputs  
            'low_toxicity': 1.0,     # Lower penalty for mildly toxic inputs
            'neutral': 0.5           # Minimal penalty for neutral inputs
        }
    
    def _get_toxicity_level(self, input_text):
        """Determine toxicity level of input text"""
        input_lower = input_text.lower()
        
        # Count forbidden words in input
        forbidden_count = sum(1 for pattern in self.forbidden_patterns 
                            if pattern.search(input_lower))
        
        if forbidden_count >= 3:
            return 'high_toxicity'
        elif forbidden_count >= 2:
            return 'medium_toxicity'
        elif forbidden_count >= 1:
            return 'low_toxicity'
        else:
            return 'neutral'
    
    def _calculate_forbidden_penalty(self, predictions, labels=None):
        """Calculate penalty with context awareness"""
        batch_size = predictions.shape[0]
        total_penalty = 0.0
        
        for i in range(batch_size):
            # Get input context (you might need to modify this based on your data structure)
            # For now, we'll use a simplified approach
            
            pred_ids = predictions[i]
            valid_pred_ids = pred_ids[pred_ids >= 0]
            
            try:
                generated_text = self.tokenizer.decode(valid_pred_ids, skip_special_tokens=True).lower()
                
                # Determine context-specific penalty
                toxicity_level = 'medium_toxicity'  # Default - you can enhance this
                context_penalty_weight = self.context_penalties.get(toxicity_level, 1.0)
                
                # Count forbidden words
                forbidden_count = 0
                for pattern in self.forbidden_patterns:
                    matches = pattern.findall(generated_text)
                    forbidden_count += len(matches)
                
                if forbidden_count > 0:
                    total_penalty += forbidden_count * context_penalty_weight
                    
            except Exception:
                continue
        
        device = predictions.device
        penalty_tensor = torch.tensor(total_penalty, dtype=torch.float32, device=device)
        return penalty_tensor

# =============================================
# TESTING UTILITIES
# =============================================
def test_custom_loss_function():
    """Test the custom loss function with sample data"""
    
    print("🧪 Testing custom loss function...")
    
    # Create a small test batch
    test_samples = [tokenized_dataset['train'][i] for i in range(2)]
    test_batch = data_collator(test_samples)
    
    # Move to model device
    model_device = next(model_lora.parameters()).device
    test_batch = {k: v.to(model_device) if isinstance(v, torch.Tensor) else v 
                  for k, v in test_batch.items()}
    
    # Test with regular trainer
    print("\n📊 Regular trainer loss:")
    regular_loss = trainer.compute_loss(model_lora, test_batch)
    print(f"Regular loss: {regular_loss.item():.4f}")
    
    # Test with custom trainer  
    print("\n🚫 Custom trainer loss:")
    custom_loss = custom_trainer.compute_loss(model_lora, test_batch)
    print(f"Custom loss: {custom_loss.item():.4f}")
    print(f"Penalty added: {(custom_loss - regular_loss).item():.4f}")
    
    return regular_loss, custom_loss

def analyze_forbidden_words_in_dataset():
    """Analyze how many forbidden words are in the training dataset"""
    
    print("📊 Analyzing forbidden words in dataset...")
    
    forbidden_in_toxic = 0
    forbidden_in_neutral = 0
    total_samples = 0
    
    # Check first 100 samples for analysis
    for i in range(min(100, len(dataset['train']))):
        sample = dataset['train'][i]
        toxic_text = sample['toxic_sentence'].lower()
        neutral_text = sample['neutral_sentence'].lower()
        
        # Count forbidden words in toxic sentences
        toxic_count = sum(1 for word in FORBIDDEN_WORDS if word in toxic_text)
        forbidden_in_toxic += toxic_count
        
        # Count forbidden words in neutral sentences  
        neutral_count = sum(1 for word in FORBIDDEN_WORDS if word in neutral_text)
        forbidden_in_neutral += neutral_count
        
        total_samples += 1
    
    print(f"\n📈 Analysis Results (first {total_samples} samples):")
    print(f"🔴 Forbidden words in toxic sentences: {forbidden_in_toxic}")
    print(f"🟢 Forbidden words in neutral sentences: {forbidden_in_neutral}")
    print(f"📉 Reduction ratio: {(forbidden_in_toxic - forbidden_in_neutral) / max(forbidden_in_toxic, 1) * 100:.1f}%")
    
    if forbidden_in_neutral > 0:
        print(f"⚠️  Warning: {forbidden_in_neutral} forbidden words found in neutral sentences!")
        print("💡 Consider reviewing your forbidden words list or dataset quality")
    
    return forbidden_in_toxic, forbidden_in_neutral

print("✅ Advanced custom loss options created!")
print("\n🎯 Available trainer types:")
print("1. CustomLossTrainer - Basic forbidden words penalty")
print("2. AdaptiveLossTrainer - Dynamic penalty that increases during training")  
print("3. ContextAwareLossTrainer - Different penalties based on input toxicity")
print("\n🧪 Testing utilities:")
print("- test_custom_loss_function() - Test loss calculation")
print("- analyze_forbidden_words_in_dataset() - Analyze dataset content")

# 🎯 Custom Loss Function Guide

## 🔧 **How It Works**

The custom loss function works by:

1. **Computing original loss** - Standard cross-entropy loss from the model
2. **Decoding predictions** - Convert model logits to text during training
3. **Detecting forbidden words** - Scan generated text for banned words using regex
4. **Adding penalty** - Add `penalty_weight × forbidden_word_count` to the loss

## 📊 **Mathematical Formula**

```
Total Loss = Original Loss + (penalty_weight × Σ forbidden_words_found)
```

**Example:**
- Original loss: `2.45`
- Forbidden words found: `2` (e.g., "stupid", "hate")
- Penalty weight: `2.0`
- **Total loss: `2.45 + (2.0 × 2) = 6.45`**

---

## 🎛️ **Configuration Options**

### **Basic Configuration**
```python
# Simple setup
custom_trainer = CustomLossTrainer(
    forbidden_words=["hate", "stupid", "toxic", "bad"],
    penalty_weight=1.5,  # Moderate penalty
    # ... other trainer arguments
)
```

### **Aggressive Penalty (Stronger Deterrent)**
```python
AGGRESSIVE_WORDS = ["hate", "kill", "die", "stupid", "idiot", "worthless"]
aggressive_trainer = CustomLossTrainer(
    forbidden_words=AGGRESSIVE_WORDS,
    penalty_weight=5.0,  # High penalty
)
```

### **Adaptive Penalty (Increases Over Time)**
```python
adaptive_trainer = AdaptiveLossTrainer(
    penalty_start=0.5,   # Start with low penalty
    penalty_end=3.0,     # End with high penalty
    forbidden_words=FORBIDDEN_WORDS
)
```

---

## ⚖️ **Penalty Weight Guidelines**

| Penalty Weight | Effect | Use Case |
|----------------|--------|----------|
| `0.1 - 0.5` | Gentle nudge | Fine-tuning already good models |
| `1.0 - 2.0` | Moderate penalty | **Recommended starting point** |
| `2.5 - 5.0` | Strong deterrent | High-toxicity datasets |
| `5.0+` | Aggressive blocking | Zero-tolerance scenarios |

---

## 🧪 **Testing & Validation**

### **Step 1: Test the Function**
```python
# Test if custom loss is working
regular_loss, custom_loss = test_custom_loss_function()
```

### **Step 2: Analyze Dataset**
```python
# Check forbidden words in your data
toxic_count, neutral_count = analyze_forbidden_words_in_dataset()
```

### **Step 3: Monitor During Training**
The trainer will log every 50 steps:
```
Step 100: Original Loss = 2.45, Forbidden Penalty = 1.20, Total Loss = 3.65
```

---

## 🎯 **Best Practices**

### **1. Start Conservative**
- Begin with `penalty_weight=1.0`
- Monitor training stability
- Increase gradually if needed

### **2. Balance is Key**
- Too high penalty → Model may avoid generating anything
- Too low penalty → Ineffective deterrent
- **Sweet spot: 1.0-2.0 for most cases**

### **3. Customize Forbidden Words**
```python
# Domain-specific forbidden words
MEDICAL_FORBIDDEN = ["kill", "die", "harm", "dangerous", "toxic"]
SOCIAL_FORBIDDEN = ["hate", "stupid", "worthless", "loser", "pathetic"]

# Combine lists
CUSTOM_FORBIDDEN = MEDICAL_FORBIDDEN + SOCIAL_FORBIDDEN
```

### **4. Monitor Training**
- Watch for loss spikes (penalty too high)
- Check generated samples during training
- Validate on test set regularly

---

## 🚀 **Quick Start Recipe**

```python
# 1. Create custom trainer
custom_trainer = CustomLossTrainer(
    model=model_lora,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    data_collator=data_collator,
    tokenizer=TOKENIZER,
    forbidden_words=FORBIDDEN_WORDS,
    penalty_weight=2.0,  # Start here
    callbacks=[debug_callback]
)

# 2. Test it works
test_custom_loss_function()

# 3. Start training
custom_trainer.train()
```

In [8]:
dataset = load_data()

# Now dataset is already a HuggingFace Dataset object
print(f"Dataset type: {type(dataset)}")
print(f"Dataset size: {len(dataset)}")
print(f"Dataset features: {dataset['train'].features}")
print(f"Sample: {dataset['train'][0]}")

INFO:__main__:Loaded 400 samples for language: en
INFO:__main__:Loaded 400 samples for language: am
INFO:__main__:Loaded 400 samples for language: am
INFO:__main__:Loaded 400 samples for language: ar
INFO:__main__:Loaded 400 samples for language: ar
INFO:__main__:Loaded 400 samples for language: de
INFO:__main__:Loaded 400 samples for language: de
INFO:__main__:Loaded 400 samples for language: es
INFO:__main__:Loaded 400 samples for language: es
INFO:__main__:Loaded 400 samples for language: hi
INFO:__main__:Loaded 400 samples for language: hi
INFO:__main__:Loaded 400 samples for language: ru
INFO:__main__:Loaded 400 samples for language: uk
INFO:__main__:Loaded 400 samples for language: ru
INFO:__main__:Loaded 400 samples for language: uk
INFO:__main__:Loaded 400 samples for language: zh
INFO:__main__:Total dataset size: 3600 samples across 9 languages
INFO:__main__:Loaded 400 samples for language: zh
INFO:__main__:Total dataset size: 3600 samples across 9 languages
INFO:__main__:Trai

Dataset type: <class 'datasets.dataset_dict.DatasetDict'>
Dataset size: 2
Dataset features: {'toxic_sentence': Value('string'), 'neutral_sentence': Value('string'), 'language': Value('string')}
Sample: {'toxic_sentence': 'Sehe ich Kathrin Göring Eckart, Merkel, Gysi, Petry, Wagenknecht und all die anderen Wendehälse die uns heute regieren. Frage ich mich. War der Fall der Mauer ein Fehler.', 'neutral_sentence': 'Sehe ich Kathrin Göring Eckart, Merkel, Gysi, Petry, Wagenknecht und all die anderen wechselnhaften  Politiker die uns heute regieren. Frage ich mich. War der Fall der Mauer ein Fehler.', 'language': 'de'}


In [11]:
import torch
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Create QLoRA configuration with bfloat16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for stability
    bnb_4bit_use_double_quant=False,        # Disabled for stability
)

# Load and quantize the model
model_quantized = T5ForConditionalGeneration.from_pretrained(
    "google/mt5-base",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# Prepare model for k-bit training
model_quantized = prepare_model_for_kbit_training(model_quantized)

# LoRA configuration
lora_config = LoraConfig(
    r=8,                    # Rank
    lora_alpha=16,          # Alpha parameter
    target_modules=["q", "v"],  # Target attention modules
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Apply LoRA to the quantized model
model_lora = get_peft_model(model_quantized, lora_config)

print("✅ QLoRA model created with bfloat16 configuration")
print(f"📊 Model type: {type(model_lora)}")
print(f"🔧 Compute dtype: {bnb_config.bnb_4bit_compute_dtype}")

# Check trainable parameters
trainable_params = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_lora.parameters())
print(f"🎯 Trainable parameters: {trainable_params:,} / {total_params:,} ({trainable_params/total_params*100:.2f}%)")

`torch_dtype` is deprecated! Use `dtype` instead!
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


✅ QLoRA model created with bfloat16 configuration
📊 Model type: <class 'peft.peft_model.PeftModelForSeq2SeqLM'>
🔧 Compute dtype: torch.bfloat16
🎯 Trainable parameters: 884,736 / 503,069,952 (0.18%)


## Default Qlora Trainer

In [25]:
# Custom callback to debug training issues
from transformers import TrainerCallback, TrainingArguments, DataCollatorForSeq2Seq
import numpy as np
from transformers import Trainer

class DebugCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 10 == 0:  # Log every 10 steps
            logs = kwargs.get('logs', {})
            loss = logs.get('train_loss', 'N/A')
            
            print(f"Step {state.global_step}: Loss = {loss}")
            
            # Check for problematic loss values
            if isinstance(loss, (int, float)):
                if loss == 0:
                    print("⚠️  WARNING: Loss is exactly 0 - check your data preprocessing!")
                elif loss < 1e-6:
                    print("⚠️  WARNING: Loss is extremely small - possible numerical issues!")
                elif np.isnan(loss):
                    print("⚠️  ERROR: Loss is NaN - stopping training recommended!")
                    control.should_training_stop = True


# First, create the tokenized dataset
def tokenize_function(examples):
    """Tokenize the dataset for T5 training"""
    # Create input text with task prefix
    input_texts = [text for text in examples['toxic_sentence']]
    target_texts = examples['neutral_sentence']
    
    # Tokenize inputs
    model_inputs = TOKENIZER(
        input_texts,
        max_length=512,
        truncation=True,
        padding=False  # We'll pad in the data collator
    )
    
    # Tokenize targets
    with TOKENIZER.as_target_tokenizer():
        labels = TOKENIZER(
            target_texts,
            max_length=512,
            truncation=True,
            padding=False
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to the train dataset
print("🔄 Tokenizing dataset...")
tokenized_dataset = dataset['train'].map(
    tokenize_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

print(f"✅ Tokenized dataset created with {len(tokenized_dataset)} samples")

# Split train into train and eval
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_dataset = DatasetDict({
    'train': train_test_split['train'],
    'eval': train_test_split['test']
})

# Add the debug callback to trainer
debug_callback = DebugCallback()

# Define training arguments
training_args = TrainingArguments(
    output_dir="./mt5-detoxify-qlora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=10,
    learning_rate=1e-4,
    bf16=True,
    logging_steps=200,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=200,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
)
# Create data collator
data_collator = DataCollatorForSeq2Seq(
    TOKENIZER, 
    model=model_lora,
    padding=True,
    return_tensors="pt"
)

# Recreate trainer with debug callback
trainer = Trainer(
    model=model_lora,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    data_collator=data_collator,
    callbacks=[debug_callback]
)

print("Trainer recreated with debug callback. Ready to train with detailed monitoring.")

🔄 Tokenizing dataset...


Map:   0%|          | 0/2880 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


✅ Tokenized dataset created with 2880 samples
Trainer recreated with debug callback. Ready to train with detailed monitoring.


In [26]:
# QUICK CHECK: Let's see what's actually in your data
print("=== QUICK DATA CHECK ===")

# Check the dataset used for tokenization
print(f"Dataset size: {len(dataset['train'])}")
print(f"Dataset features: {dataset['train'].features}")

# Check a few raw samples
for i in range(3):
    sample = dataset['train'][i]
    toxic = sample['toxic_sentence']
    neutral = sample['neutral_sentence']
    
    print(f"\nSample {i}:")
    print(f"Toxic  : '{toxic[:80]}{'...' if len(toxic) > 80 else ''}'")
    print(f"Neutral: '{neutral[:80]}{'...' if len(neutral) > 80 else ''}'")
    print(f"Same? {toxic == neutral}")
    
    if toxic == neutral:
        print("🚨 PROBLEM: Toxic and neutral sentences are identical!")
    
# Check tokenized samples
print(f"\n=== TOKENIZED CHECK ===")
tokenized_train_dataset = tokenized_dataset['train']
for i in range(2):
    tokenized_sample = tokenized_train_dataset[i]
    
    # Decode back to text to see what was actually tokenized
    input_text = TOKENIZER.decode(tokenized_sample['input_ids'], skip_special_tokens=True)
    label_text = TOKENIZER.decode(tokenized_sample['labels'], skip_special_tokens=True)
    
    print(f"\nTokenized Sample {i}:")
    print(f"Input : '{input_text}'")
    print(f"Label : '{label_text}'")
    print(f"Same? {input_text == label_text}")

print("=== END QUICK CHECK ===")

=== QUICK DATA CHECK ===
Dataset size: 2880
Dataset features: {'toxic_sentence': Value('string'), 'neutral_sentence': Value('string'), 'language': Value('string')}

Sample 0:
Toxic  : 'Sehe ich Kathrin Göring Eckart, Merkel, Gysi, Petry, Wagenknecht und all die and...'
Neutral: 'Sehe ich Kathrin Göring Eckart, Merkel, Gysi, Petry, Wagenknecht und all die and...'
Same? False

Sample 1:
Toxic  : 'THIS SCHOOL DISTRICT SUCKS i know cause i go there'
Neutral: 'This school district is unsatifactory i know cause i go there'
Same? False

Sample 2:
Toxic  : 'ሰይጣን እባክህን ይህንን ኃጢአተኛ ይቅር በለው ፣ እሱ የሚናገረውን አያውቅም'
Neutral: 'እባክህን ይህንን ምስኪን ይቅር በለው ፣ ባለማወቅ ነው የተናገረው'
Same? False

=== TOKENIZED CHECK ===

Tokenized Sample 0:
Input : 'አንተ ዶማ የሚያሳዝነው ሌላ የትግራይና የሀገራችን ትውልድ ለናንተ ዐይነቱ ውሻ ብሎ መሞቱና አካሉን ማጣቱ ነው የሚያሳዝነው።'
Label : 'የሚያሳዝነው ሌላ የትግራይና የሀገራችን ትውልድ ለጥቂቶቸ ተብሎ መሞቱና አካሉን ማጣቱ ነው የሚያሳዝነው።'
Same? False

Tokenized Sample 1:
Input : 'Тинка ходи на Jazz fest і вино післяЖінку свою веди блять.'
Label : 'Тинка х

In [27]:
# TARGETED DEBUG: Check what DataCollator is actually doing
print("=== DATACOLLATOR DEBUG ===")

# Get a few samples
samples = [tokenized_train_dataset[i] for i in range(2)]

print("Before DataCollator:")
for i, sample in enumerate(samples):
    print(f"\nSample {i}:")
    print(f"  Input IDs (first 10): {sample['input_ids'][:10]}")
    print(f"  Labels (first 10): {sample['labels'][:10]}")

# Apply DataCollator
batch = data_collator(samples)

print(f"\nAfter DataCollator:")
print(f"Batch keys: {batch.keys()}")
print(f"Input shape: {batch['input_ids'].shape}")
print(f"Labels shape: {batch['labels'].shape}")

# Check first sample in detail
input_ids_0 = batch['input_ids'][0]
labels_0 = batch['labels'][0]

print(f"\nDetailed check of first sample:")
print(f"Input IDs (first 15): {input_ids_0[:15].tolist()}")
print(f"Labels (first 15): {labels_0[:15].tolist()}")

# Count valid vs ignored labels
valid_labels = (labels_0 != -100).sum().item()
total_labels = len(labels_0)
print(f"Valid labels: {valid_labels}/{total_labels} ({valid_labels/total_labels*100:.1f}%)")

# Check if there's any real difference
if valid_labels > 0:
    # Find where valid labels start and compare
    valid_mask = labels_0 != -100
    valid_positions = torch.where(valid_mask)[0]
    
    if len(valid_positions) > 0:
        start_pos = valid_positions[0].item()
        end_pos = min(start_pos + 10, len(labels_0))
        
        input_segment = input_ids_0[start_pos:end_pos]
        label_segment = labels_0[start_pos:end_pos]
        
        print(f"\nComparison at position {start_pos}-{end_pos}:")
        print(f"Input segment: {input_segment.tolist()}")
        print(f"Label segment: {label_segment.tolist()}")
        
        # Decode these segments
        try:
            input_text = TOKENIZER.decode(input_segment, skip_special_tokens=True)
            label_text = TOKENIZER.decode(label_segment, skip_special_tokens=True) 
            print(f"Input text: '{input_text}'")
            print(f"Label text: '{label_text}'")
            
            are_identical = torch.equal(input_segment, label_segment)
            print(f"Segments identical? {are_identical}")
            
            if are_identical:
                print("🚨 FOUND THE PROBLEM: DataCollator is making input and labels identical!")
            
        except Exception as e:
            print(f"Decode error: {e}")

print("=== END DATACOLLATOR DEBUG ===")

=== DATACOLLATOR DEBUG ===
Before DataCollator:

Sample 0:
  Input IDs (first 10): [19293, 5359, 259, 12549, 3866, 32241, 3562, 3858, 51663, 259]
  Labels (first 10): [32241, 3562, 3858, 51663, 259, 7234, 2423, 259, 24637, 41782]

Sample 1:
  Input IDs (first 10): [1454, 68588, 259, 67303, 310, 34792, 12065, 259, 389, 75376]
  Labels (first 10): [1454, 68588, 259, 67303, 310, 34792, 12065, 259, 389, 75376]

After DataCollator:
Batch keys: KeysView({'input_ids': tensor([[19293,  5359,   259, 12549,  3866, 32241,  3562,  3858, 51663,   259,
          7234,  2423,   259, 24637, 41782, 87090,  1237, 29795, 51301, 19593,
           259,   986,  1984,  1305,  2308,  7958,  1791, 90748,   259, 66631,
          2226, 73354,   259,  1984, 11258,  4494,  6304,  2338,  5293,  5644,
          1791,  1876,  2401, 80344,  5218,  4580,  5644,  4476, 32241,  3562,
          3858, 51663,  3369,     1],
        [ 1454, 68588,   259, 67303,   310, 34792, 12065,   259,   389, 75376,
         15322,  3283,

In [28]:
# FINAL DEBUG: Check model and training setup
print("=== MODEL & TRAINING DEBUG ===")

# 1. Check if LoRA is actually applied
print("LoRA Configuration:")
print(f"Model type: {type(model_lora)}")
print(f"Trainable parameters:")
trainable_params = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_lora.parameters())
print(f"  Trainable: {trainable_params:,}")
print(f"  Total: {total_params:,}")
print(f"  Percentage: {trainable_params/total_params*100:.2f}%")

# 2. Test forward pass manually
print(f"\n=== MANUAL FORWARD PASS TEST ===")
model_lora.eval()

# Get a small batch
test_batch = data_collator([tokenized_train_dataset[0], tokenized_train_dataset[1]])

print(f"Test batch shapes:")
print(f"  Input IDs: {test_batch['input_ids'].shape}")
print(f"  Labels: {test_batch['labels'].shape}")

# Get model device
model_device = next(model_lora.parameters()).device
print(f"Model device: {model_device}")

# Move batch to same device as model
test_batch = {k: v.to(model_device) if isinstance(v, torch.Tensor) else v for k, v in test_batch.items()}
print(f"Moved batch to device: {model_device}")

# Manual forward pass
with torch.no_grad():
    outputs = model_lora(
        input_ids=test_batch['input_ids'],
        attention_mask=test_batch['attention_mask'],
        labels=test_batch['labels']
    )
    
    print(f"\nModel outputs:")
    print(f"  Loss: {outputs.loss}")
    print(f"  Loss item: {outputs.loss.item()}")
    print(f"  Logits shape: {outputs.logits.shape}")
    
    # Check if loss is actually zero
    if outputs.loss.item() == 0.0:
        print("🚨 CONFIRMED: Model is returning exactly 0 loss!")
        
        # Check logits and labels in detail
        logits = outputs.logits
        labels = test_batch['labels']
        
        # Manual loss calculation
        loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
        manual_loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        print(f"  Manual loss calculation: {manual_loss.item()}")
        
        # Check if all labels are being ignored
        valid_labels = (labels != -100).sum()
        print(f"  Valid labels in batch: {valid_labels}")
        
        if valid_labels == 0:
            print("🚨 PROBLEM: All labels are being ignored (-100)!")
        else:
            print("🤔 MYSTERY: Labels are valid but loss is still 0...")
            
            # Check if model weights are frozen
            print(f"\nChecking if model weights are trainable:")
            for name, param in model_lora.named_parameters():
                if param.requires_grad:
                    print(f"  {name}: requires_grad=True, shape={param.shape}")
                    break
            else:
                print("🚨 PROBLEM: No trainable parameters found!")
    else:
        print(f"✅ Loss is normal: {outputs.loss.item()}")

model_lora.train()  # Back to training mode
print("=== END MODEL DEBUG ===")

=== MODEL & TRAINING DEBUG ===
LoRA Configuration:
Model type: <class 'peft.peft_model.PeftModelForSeq2SeqLM'>
Trainable parameters:
  Trainable: 884,736
  Total: 503,069,952
  Percentage: 0.18%

=== MANUAL FORWARD PASS TEST ===
Test batch shapes:
  Input IDs: torch.Size([2, 54])
  Labels: torch.Size([2, 45])
Model device: cuda:0
Moved batch to device: cuda:0

Model outputs:
  Loss: 3.701512098312378
  Loss item: 3.701512098312378
  Logits shape: torch.Size([2, 45, 250112])
✅ Loss is normal: 3.701512098312378
=== END MODEL DEBUG ===


In [29]:
# EXTENDED DEBUG: Investigate NaN loss issue
print("=== NaN LOSS INVESTIGATION ===")

# Get a small batch for detailed analysis
test_batch = data_collator([tokenized_train_dataset[0]])
model_device = next(model_lora.parameters()).device
test_batch = {k: v.to(model_device) if isinstance(v, torch.Tensor) else v for k, v in test_batch.items()}

model_lora.eval()
with torch.no_grad():
    # Check input ranges
    input_ids = test_batch['input_ids']
    labels = test_batch['labels']
    
    print(f"Input analysis:")
    print(f"  Input IDs range: {input_ids.min().item()} - {input_ids.max().item()}")
    print(f"  Vocab size: {TOKENIZER.vocab_size}")
    print(f"  Model vocab size: {model_lora.config.vocab_size}")
    
    # Check if any input IDs are out of vocab range
    max_id = input_ids.max().item()
    vocab_size = model_lora.config.vocab_size
    if max_id >= vocab_size:
        print(f"🚨 PROBLEM: Input ID {max_id} >= vocab size {vocab_size}")
    
    print(f"\nLabel analysis:")
    valid_labels = labels[labels != -100]
    if len(valid_labels) > 0:
        print(f"  Valid labels range: {valid_labels.min().item()} - {valid_labels.max().item()}")
        max_label = valid_labels.max().item()
        if max_label >= vocab_size:
            print(f"🚨 PROBLEM: Label {max_label} >= vocab size {vocab_size}")
    else:
        print("  No valid labels found!")
    
    # Check model embedding layer
    print(f"\nModel embedding analysis:")
    with torch.no_grad():
        # Get embeddings for a small range of tokens
        try:
            sample_embeddings = model_lora.base_model.model.shared(torch.tensor([0, 1, 2]).to(model_device))
            print(f"  Sample embeddings shape: {sample_embeddings.shape}")
            print(f"  Sample embeddings contain NaN: {torch.isnan(sample_embeddings).any().item()}")
            print(f"  Sample embeddings contain Inf: {torch.isinf(sample_embeddings).any().item()}")
        except Exception as e:
            print(f"  Error accessing embeddings: {e}")
    
    # Forward pass with gradient checking
    try:
        outputs = model_lora(
            input_ids=input_ids,
            attention_mask=test_batch['attention_mask'],
            labels=labels
        )
        
        logits = outputs.logits
        print(f"\nLogits analysis:")
        print(f"  Logits shape: {logits.shape}")
        print(f"  Logits contain NaN: {torch.isnan(logits).any().item()}")
        print(f"  Logits contain Inf: {torch.isinf(logits).any().item()}")
        print(f"  Logits range: {logits.min().item():.3f} - {logits.max().item():.3f}")
        
        # Manual cross-entropy calculation with detailed checks
        print(f"\nManual loss calculation:")
        loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100, reduction='none')
        
        # Reshape for loss calculation
        shift_logits = logits.view(-1, logits.size(-1))
        shift_labels = labels.view(-1)
        
        print(f"  Shift logits shape: {shift_logits.shape}")
        print(f"  Shift labels shape: {shift_labels.shape}")
        
        # Calculate loss per token
        token_losses = loss_fct(shift_logits, shift_labels)
        valid_losses = token_losses[shift_labels != -100]
        
        print(f"  Valid token losses: {len(valid_losses)}")
        if len(valid_losses) > 0:
            print(f"  Token losses contain NaN: {torch.isnan(valid_losses).any().item()}")
            print(f"  Token losses contain Inf: {torch.isinf(valid_losses).any().item()}")
            print(f"  Token losses range: {valid_losses.min().item():.3f} - {valid_losses.max().item():.3f}")
            
            mean_loss = valid_losses.mean()
            print(f"  Mean loss: {mean_loss.item()}")
        
    except Exception as e:
        print(f"Forward pass error: {e}")

model_lora.train()
print("=== END NaN INVESTIGATION ===")

=== NaN LOSS INVESTIGATION ===
Input analysis:
  Input IDs range: 1 - 90748
  Vocab size: 250100
  Model vocab size: 250112

Label analysis:
  Valid labels range: 1 - 87090

Model embedding analysis:
  Sample embeddings shape: torch.Size([3, 768])
  Sample embeddings contain NaN: False
  Sample embeddings contain Inf: False

Logits analysis:
  Logits shape: torch.Size([1, 45, 250112])
  Logits contain NaN: False
  Logits contain Inf: False
  Logits range: -66.000 - 5.062

Manual loss calculation:
  Shift logits shape: torch.Size([45, 250112])
  Shift labels shape: torch.Size([45])
  Valid token losses: 45
  Token losses contain NaN: False
  Token losses contain Inf: False
  Token losses range: 0.098 - 26.500
  Mean loss: 3.417591094970703
=== END NaN INVESTIGATION ===


In [30]:
trainer.train()

Step,Training Loss,Validation Loss
200,6.174100,2.758302
400,3.690300,2.345390
600,3.131000,2.206075
800,2.905000,2.126004
1000,2.824300,2.068775
1200,2.741000,2.021767
1400,2.693800,2.008225


Step 10: Loss = N/A
Step 20: Loss = N/A
Step 20: Loss = N/A
Step 30: Loss = N/A
Step 30: Loss = N/A
Step 40: Loss = N/A
Step 40: Loss = N/A
Step 50: Loss = N/A
Step 50: Loss = N/A
Step 60: Loss = N/A
Step 60: Loss = N/A
Step 70: Loss = N/A
Step 70: Loss = N/A
Step 80: Loss = N/A
Step 80: Loss = N/A
Step 90: Loss = N/A
Step 90: Loss = N/A
Step 100: Loss = N/A
Step 100: Loss = N/A
Step 110: Loss = N/A
Step 110: Loss = N/A
Step 120: Loss = N/A
Step 120: Loss = N/A
Step 130: Loss = N/A
Step 130: Loss = N/A
Step 140: Loss = N/A
Step 140: Loss = N/A
Step 150: Loss = N/A
Step 150: Loss = N/A
Step 160: Loss = N/A
Step 160: Loss = N/A
Step 170: Loss = N/A
Step 170: Loss = N/A
Step 180: Loss = N/A
Step 180: Loss = N/A
Step 190: Loss = N/A
Step 190: Loss = N/A
Step 200: Loss = N/A
Step 200: Loss = N/A
Step 210: Loss = N/A
Step 210: Loss = N/A
Step 220: Loss = N/A
Step 220: Loss = N/A
Step 230: Loss = N/A
Step 230: Loss = N/A
Step 240: Loss = N/A
Step 240: Loss = N/A
Step 250: Loss = N/A
Step 250:

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Step 510: Loss = N/A
Step 520: Loss = N/A
Step 520: Loss = N/A
Step 530: Loss = N/A
Step 530: Loss = N/A
Step 540: Loss = N/A
Step 540: Loss = N/A
Step 550: Loss = N/A
Step 550: Loss = N/A
Step 560: Loss = N/A
Step 560: Loss = N/A
Step 570: Loss = N/A
Step 570: Loss = N/A
Step 580: Loss = N/A
Step 580: Loss = N/A
Step 590: Loss = N/A
Step 590: Loss = N/A
Step 600: Loss = N/A
Step 600: Loss = N/A
Step 610: Loss = N/A
Step 610: Loss = N/A
Step 620: Loss = N/A
Step 620: Loss = N/A
Step 630: Loss = N/A
Step 630: Loss = N/A
Step 640: Loss = N/A
Step 640: Loss = N/A
Step 650: Loss = N/A
Step 650: Loss = N/A
Step 660: Loss = N/A
Step 660: Loss = N/A
Step 670: Loss = N/A
Step 670: Loss = N/A
Step 680: Loss = N/A
Step 680: Loss = N/A
Step 690: Loss = N/A
Step 690: Loss = N/A
Step 700: Loss = N/A
Step 700: Loss = N/A
Step 710: Loss = N/A
Step 710: Loss = N/A
Step 720: Loss = N/A
Step 720: Loss = N/A
Step 730: Loss = N/A
Step 730: Loss = N/A
Step 740: Loss = N/A
Step 740: Loss = N/A
Step 750: Los

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Step 1010: Loss = N/A
Step 1020: Loss = N/A
Step 1020: Loss = N/A
Step 1030: Loss = N/A
Step 1030: Loss = N/A
Step 1040: Loss = N/A
Step 1040: Loss = N/A
Step 1050: Loss = N/A
Step 1050: Loss = N/A
Step 1060: Loss = N/A
Step 1060: Loss = N/A
Step 1070: Loss = N/A
Step 1070: Loss = N/A
Step 1080: Loss = N/A
Step 1080: Loss = N/A
Step 1090: Loss = N/A
Step 1090: Loss = N/A
Step 1100: Loss = N/A
Step 1100: Loss = N/A
Step 1110: Loss = N/A
Step 1110: Loss = N/A
Step 1120: Loss = N/A
Step 1120: Loss = N/A
Step 1130: Loss = N/A
Step 1130: Loss = N/A
Step 1140: Loss = N/A
Step 1140: Loss = N/A
Step 1150: Loss = N/A
Step 1150: Loss = N/A
Step 1160: Loss = N/A
Step 1160: Loss = N/A
Step 1170: Loss = N/A
Step 1170: Loss = N/A
Step 1180: Loss = N/A
Step 1180: Loss = N/A
Step 1190: Loss = N/A
Step 1190: Loss = N/A
Step 1200: Loss = N/A
Step 1200: Loss = N/A
Step 1210: Loss = N/A
Step 1210: Loss = N/A
Step 1220: Loss = N/A
Step 1220: Loss = N/A
Step 1230: Loss = N/A
Step 1230: Loss = N/A
Step 1240:

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


TrainOutput(global_step=1440, training_loss=3.4316887113783094, metrics={'train_runtime': 4488.4579, 'train_samples_per_second': 5.133, 'train_steps_per_second': 0.321, 'total_flos': 1833619651448832.0, 'train_loss': 3.4316887113783094, 'epoch': 10.0})

In [31]:
# apply the inference in the test dataset
# 🔮 INFERENCE ON TEST DATASET

print("🔮 Starting inference on test dataset...")

# Set model to evaluation mode
model_lora.eval()

# Get test dataset
test_dataset = dataset['test']
# Filter test dataset to only include Spanish samples
test_dataset_es = test_dataset.filter(lambda x: x['language'] == 'es')
print(f"📊 Spanish test dataset size: {len(test_dataset_es)} samples")
print(f"📊 Original test dataset size: {len(test_dataset)} samples")

# Update test_dataset to use the filtered version
test_dataset = test_dataset_es

# Function to generate detoxified text
def detoxify_text(toxic_text, max_length=512):
    """Generate detoxified version of toxic text"""
    # Add task prefix
    input_text = f"{toxic_text}"
    
    # Tokenize input
    inputs = TOKENIZER(
        input_text,
        return_tensors="pt",
        max_length=max_length,
        truncation=True,
        padding=True
    ).to(model_device)
    
    # Generate output
    with torch.no_grad():
        outputs = model_lora.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_beams=4,
            do_sample=False,
            early_stopping=True,
            pad_token_id=TOKENIZER.pad_token_id,
            eos_token_id=TOKENIZER.eos_token_id
        )
    
    # Decode output
    generated_text = TOKENIZER.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Run inference on test samples
results = []
num_samples = min(10, len(test_dataset))  # Test on first 10 samples

print(f"\n🧪 Testing on {num_samples} samples:")
print("=" * 80)

for i in range(num_samples):
    sample = test_dataset[i]
    toxic_sentence = sample['toxic_sentence']
    expected_neutral = sample['neutral_sentence']
    language = sample['language']
    
    # Generate detoxified version
    try:
        generated_neutral = detoxify_text(toxic_sentence)
        
        # Store results
        result = {
            'index': i,
            'language': language,
            'toxic_input': toxic_sentence,
            'expected_output': expected_neutral,
            'generated_output': generated_neutral
        }
        results.append(result)
        
        # Print results
        print(f"\nSample {i+1} ({language}):")
        print(f"🔴 Toxic:     {toxic_sentence[:100]}{'...' if len(toxic_sentence) > 100 else ''}")
        print(f"🟢 Expected:  {expected_neutral[:100]}{'...' if len(expected_neutral) > 100 else ''}")
        print(f"🔵 Generated: {generated_neutral[:100]}{'...' if len(generated_neutral) > 100 else ''}")
        
    except Exception as e:
        print(f"❌ Error processing sample {i}: {e}")
        continue

print("=" * 80)
print(f"✅ Inference completed on {len(results)} samples")


🔮 Starting inference on test dataset...
📊 Spanish test dataset size: 70 samples
📊 Original test dataset size: 720 samples

🧪 Testing on 10 samples:

Sample 1 (es):
🔴 Toxic:     Que analogía rebuscada y de mierda
🟢 Expected:  Qué analogía confusa
🔵 Generated: <extra_id_0> Que analogía rebuscada y de mierda

Sample 1 (es):
🔴 Toxic:     Que analogía rebuscada y de mierda
🟢 Expected:  Qué analogía confusa
🔵 Generated: <extra_id_0> Que analogía rebuscada y de mierda

Sample 2 (es):
🔴 Toxic:     Debería darte vergüenza ofender las creencias de otras persona, intolerante fascista.
🟢 Expected:  Debería darte vergüenza ofender las creencias de otras persona.
🔵 Generated: <extra_id_0> debería darte vergüenza ofender las creencias de otras personas, intolerante fascista.

Sample 2 (es):
🔴 Toxic:     Debería darte vergüenza ofender las creencias de otras persona, intolerante fascista.
🟢 Expected:  Debería darte vergüenza ofender las creencias de otras persona.
🔵 Generated: <extra_id_0> debería dar

In [32]:
import pandas as pd
from tqdm import tqdm
import os
from datetime import datetime

# 🔮 FULL DATASET INFERENCE & CSV EXPORT


print("🔮 Starting full dataset inference...")

# Set model to evaluation mode
model_lora.eval()

# Get the full test dataset (all languages)
full_test_dataset = dataset['test']
print(f"📊 Full test dataset size: {len(full_test_dataset)} samples")

# Function to generate detoxified text (reuse from previous cell)
def detoxify_text(toxic_text, max_length=512):
    """Generate detoxified version of toxic text"""
    input_text = f"{toxic_text}"
    
    inputs = TOKENIZER(
        input_text,
        return_tensors="pt",
        max_length=max_length,
        truncation=True,
        padding=True
    ).to(model_device)
    
    with torch.no_grad():
        outputs = model_lora.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_beams=4,
            do_sample=False,
            early_stopping=True,
            pad_token_id=TOKENIZER.pad_token_id,
            eos_token_id=TOKENIZER.eos_token_id
        )
    
    generated_text = TOKENIZER.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Prepare results list
all_results = []

# Process all samples with progress bar
print(f"\n🧪 Processing {len(full_test_dataset)} samples...")
for i in tqdm(range(len(full_test_dataset)), desc="Generating detoxified text"):
    sample = full_test_dataset[i]
    
    try:
        # Extract sample data
        toxic_sentence = sample['toxic_sentence']
        expected_neutral = sample['neutral_sentence']
        language = sample['language']
        
        # Generate detoxified version
        generated_neutral = detoxify_text(toxic_sentence)
        
        # Store results
        result = {
            'sample_id': i,
            'language': language,
            'toxic_input': toxic_sentence,
            'expected_output': expected_neutral,
            'generated_output': generated_neutral
        }
        all_results.append(result)
        
        # Print progress every 100 samples
        if (i + 1) % 100 == 0:
            print(f"✅ Processed {i + 1}/{len(full_test_dataset)} samples")
            
    except Exception as e:
        print(f"❌ Error processing sample {i}: {e}")
        # Add error entry
        all_results.append({
            'sample_id': i,
            'language': sample.get('language', 'unknown'),
            'toxic_input': sample.get('toxic_sentence', ''),
            'expected_output': sample.get('neutral_sentence', ''),
            'generated_output': f'ERROR: {str(e)}'
        })
        continue

# Create DataFrame
results_df = pd.DataFrame(all_results)

# Add timestamp for unique filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"mt5_detoxification_results_{timestamp}.csv"
# Save to CSV
results_df.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"\n✅ Inference completed!")
print(f"📊 Results summary:")
print(f"  Total samples processed: {len(results_df)}")
print(f"  Languages in dataset: {results_df['language'].value_counts().to_dict()}")
print(f"  Successful generations: {len(results_df[~results_df['generated_output'].str.contains('ERROR:')])}")
print(f"  Errors: {len(results_df[results_df['generated_output'].str.contains('ERROR:')])}")
print(f"  📁 Results saved to: {csv_filename}")

# Display sample results
print(f"\n🎯 Sample results:")
print("=" * 100)
for i in range(min(5, len(results_df))):
    row = results_df.iloc[i]
    print(f"\nSample {i+1} ({row['language']}):")
    print(f"🔴 Toxic:     {row['toxic_input'][:80]}{'...' if len(row['toxic_input']) > 80 else ''}")
    print(f"🟢 Expected:  {row['expected_output'][:80]}{'...' if len(row['expected_output']) > 80 else ''}")
    print(f"🔵 Generated: {row['generated_output'][:80]}{'...' if len(row['generated_output']) > 80 else ''}")

# Basic evaluation metrics
print(f"\n📈 Quick evaluation metrics:")

# Language-wise statistics
lang_stats = results_df.groupby('language').agg({
    'sample_id': 'count',
    'generated_output': lambda x: (~x.str.contains('ERROR:')).sum()
}).rename(columns={'sample_id': 'total_samples', 'generated_output': 'successful_generations'})

print("\n📊 Per-language statistics:")
for lang, stats in lang_stats.iterrows():
    success_rate = (stats['successful_generations'] / stats['total_samples']) * 100
    print(f"  {lang}: {stats['successful_generations']}/{stats['total_samples']} ({success_rate:.1f}% success)")

print(f"\n💾 CSV file '{csv_filename}' contains all results and can be opened in Excel or imported for further analysis.")

with open(csv_filename) as f:
    content = f.read()

print(content)

🔮 Starting full dataset inference...
📊 Full test dataset size: 720 samples

🧪 Processing 720 samples...


Generating detoxified text:  14%|█▍        | 100/720 [02:20<15:09,  1.47s/it]

✅ Processed 100/720 samples


Generating detoxified text:  28%|██▊       | 200/720 [04:48<14:02,  1.62s/it]

✅ Processed 200/720 samples


...

✅ Processed 600/720 samples


Generating detoxified text:  97%|█████████▋| 700/720 [19:39<00:26,  1.34s/it]

✅ Processed 700/720 samples


Generating detoxified text: 100%|██████████| 720/720 [20:05<00:00,  1.67s/it]


✅ Inference completed!
📊 Results summary:
  Total samples processed: 720
  Languages in dataset: {'am': 89, 'uk': 88, 'en': 86, 'zh': 83, 'ar': 79, 'ru': 76, 'de': 76, 'hi': 73, 'es': 70}
  Successful generations: 720
  Errors: 0
  📁 Results saved to: mt5_detoxification_results_20251124_194544.csv

🎯 Sample results:

Sample 1 (ru):
🔴 Toxic:     ну и дурак ! нужно россию присоеденнить к белорусии - лучщеб было !
🟢 Expected:  Нужно Россию присоединить к Белорусии лучше бы было
🔵 Generated: ну и дурак ! нужно россию присоеденнить к белорусии - лучщеб было !

Sample 2 (am):
🔴 Toxic:     አንተ ደንቆሮ የሸርሙጣ ልጅ አህያ ቦዶ ጭንቅላት ዘረኛ የሴት ልጅ አቅምህን እውቅ እንዳ አንተ አይነቱ ተላላኪ ነው አገራችን እ...
🟢 Expected:  አቅምህን ብታውቅ ደህና ነው ፤ እናንተ ናችሁ ባንድ በኩል ሀገሪቱ እንዳትረጋጋ ያደረጋችኋት
🔵 Generated: ደንቆሮ የሴት ልጅ አቅምህን እውቅ እንዳ አንተ አይነቱ ተላላኪ ነው አገራችን እንዳትረጋጋ ያደረጋት ቆሻሻ

Sample 3 (ru):
🔴 Toxic:     везде бабушки блядь а дедушки ведь она пиздует к внукам а дед ебашет на работе и...
🟢 Expected:  везде бабушки и дедушки: бабушка ездит к внукам,

## Saving

In [52]:
# 💾 SAVE TRAINED MODEL - Multiple Options (with Cloud Storage support)

print("=== SAVING FINE-TUNED mT5 MODEL ===")

# Configure save path with automatic cloud storage detection
import os
from datetime import datetime

# Create timestamped folder name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_folder_name = f"mt5-detox-lora-{timestamp}"

# Smart path detection function
def get_best_save_path():
    """Automatically detect the best cloud storage path available"""
    username = os.getenv('USERNAME', 'user')  # Windows
    
    # Priority order: OneDrive -> Google Drive -> Local
    possible_paths = [
        # OneDrive paths (since you're using OneDrive)
        f"C:\\Users\\{username}\\OneDrive\\ML_Models\\mt5-detox\\",
        f"C:\\Users\\{username}\\OneDrive\\Documents\\ML_Models\\mt5-detox\\",
        
        # Google Drive Desktop paths
        f"C:\\Users\\{username}\\Google Drive\\ML_Models\\mt5-detox\\",
        "G:\\My Drive\\ML_Models\\mt5-detox\\",
        
        # Local fallback
        ".\\model_backups\\",
    ]
    
    for path in possible_paths:
        try:
            # Test if parent directory exists or can be created
            parent = os.path.dirname(path)
            if os.path.exists(parent) or path.startswith('.\\'):
                os.makedirs(path, exist_ok=True)
                return path, "cloud" if not path.startswith('.\\') else "local"
        except:
            continue
    
    # Final fallback
    fallback = ".\\model_backups\\"
    os.makedirs(fallback, exist_ok=True)
    return fallback, "local"

# Get the best available path
base_path, storage_type = get_best_save_path()
save_path = os.path.join(base_path, model_folder_name)

print(f"📁 Storage type: {storage_type.upper()}")
print(f"💾 Save path: {save_path}")

# Create directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)
print(f"📁 Created save directory: {save_path}")

# OPTION 1: Save LoRA adapter to Google Drive (Recommended)
print("\n🔧 Option 1: Save LoRA adapter to Google Drive")
try:
    model_lora_stable.save_pretrained(save_path)
    print(f"✅ LoRA adapter saved to: {save_path}")
    
    # Save tokenizer as well (important!)
    print("📝 Saving tokenizer...")
    TOKENIZER.save_pretrained(save_path)
    print("✅ Tokenizer saved alongside the model")
    
    # Create a model info file
    info_file = os.path.join(save_path, "model_info.txt")
    with open(info_file, "w") as f:
        f.write(f"Model Training Information\n")
        f.write(f"========================\n")
        f.write(f"Training Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Base Model: google/mt5-base\n")
        f.write(f"Task: Text Detoxification\n")
        f.write(f"LoRA Rank: {model_lora_stable.peft_config['default'].r}\n")
        f.write(f"LoRA Alpha: {model_lora_stable.peft_config['default'].lora_alpha}\n")
        f.write(f"Target Modules: {model_lora_stable.peft_config['default'].target_modules}\n")
        f.write(f"Training Precision: bfloat16\n")
        f.write(f"Dataset Size: {len(tokenized_dataset)} samples\n")
    print(f"📄 Model info saved to: {info_file}")
    
except Exception as e:
    print(f"❌ Error saving to Google Drive: {e}")
    print("💡 Falling back to local save...")
    fallback_path = f"./mt5-detox-lora-{timestamp}"
    model_lora_stable.save_pretrained(fallback_path)
    TOKENIZER.save_pretrained(fallback_path)
    print(f"✅ Model saved locally to: {fallback_path}")

# OPTION 2: Additional backup to Hugging Face Hub
print("\n☁️ Option 2: Backup to Hugging Face Hub (optional)")
print("To save to HF Hub, uncomment and run:")
print("# model_lora_stable.push_to_hub('your-username/mt5-detox-model')")

print(f"\n📊 Model info:")
print(f"  Model type: {type(model_lora_stable)}")
print(f"  Base model: google/mt5-base")
print(f"  LoRA rank: {model_lora_stable.peft_config['default'].r}")
print(f"  Target modules: {model_lora_stable.peft_config['default'].target_modules}")
print(f"  Training precision: bfloat16")
print(f"  Save location: {save_path}")

print(f"\n🎯 Model successfully saved to Google Drive with timestamp: {timestamp}")
print("💡 The timestamped folder helps you keep track of different training runs!")

=== SAVING FINE-TUNED mT5 MODEL ===
📁 Storage type: LOCAL
💾 Save path: .\model_backups\/mt5-detox-lora-20251122_161504
📁 Created save directory: .\model_backups\/mt5-detox-lora-20251122_161504

🔧 Option 1: Save LoRA adapter to Google Drive


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


✅ LoRA adapter saved to: .\model_backups\/mt5-detox-lora-20251122_161504
📝 Saving tokenizer...
✅ Tokenizer saved alongside the model
📄 Model info saved to: .\model_backups\/mt5-detox-lora-20251122_161504/model_info.txt

☁️ Option 2: Backup to Hugging Face Hub (optional)
To save to HF Hub, uncomment and run:
# model_lora_stable.push_to_hub('your-username/mt5-detox-model')

📊 Model info:
  Model type: <class 'peft.peft_model.PeftModelForSeq2SeqLM'>
  Base model: google/mt5-base
  LoRA rank: 8
  Target modules: {'q', 'v'}
  Training precision: bfloat16
  Save location: .\model_backups\/mt5-detox-lora-20251122_161504

🎯 Model successfully saved to Google Drive with timestamp: 20251122_161504
💡 The timestamped folder helps you keep track of different training runs!


# 📂 **Model Saving Options Explained**

## 🔧 **Option 1: LoRA Adapter Only (Recommended)**
- **Size**: ~10-50 MB (very small!)
- **Contains**: Only the fine-tuned LoRA weights
- **Requires**: Original base model (google/mt5-base) to load
- **Best for**: Sharing, version control, experimentation

## 🏗️ **Option 2: Complete Model** 
- **Size**: ~2-4 GB (full model size)
- **Contains**: Entire model with merged weights
- **Standalone**: Can be loaded independently
- **Issues**: May not work with quantized models

## ☁️ **Option 3: Hugging Face Hub**
- **Benefits**: Easy sharing, version control, public access
- **Size**: Same as Option 1 (just LoRA adapter)
- **Usage**: Perfect for model deployment and collaboration

---

# 🎯 **Why LoRA Adapter is Best**

1. **Tiny file size** - Only stores the adaptation, not the full model
2. **Version control friendly** - Easy to track changes in Git
3. **Multiple adapters** - Can train different adapters for different tasks
4. **Memory efficient** - Load base model once, swap adapters quickly
5. **Quantization compatible** - Works perfectly with your 4-bit setup

---

# 📁 **File Structure After Saving**

```
mt5-detox-lora-adapter/
├── adapter_config.json       # LoRA configuration
├── adapter_model.safetensors  # LoRA weights (~10-50MB)
├── tokenizer.json            # Tokenizer files
├── tokenizer_config.json
├── special_tokens_map.json
└── spiece.model
```

**Total size**: Usually under 100MB vs ~2GB for full model!

In [ ]:
# 🔧 GOOGLE DRIVE SETUP HELPER

def setup_google_drive_path():
    """Helper function to set up cloud storage path based on local environment"""
    import os
    from pathlib import Path
    
    username = os.getenv('USERNAME', 'user')  # Windows
    
    # Priority order paths for local Windows environment
    possible_paths = [
        # OneDrive paths (highest priority since you're using OneDrive)
        f"C:\\Users\\{username}\\OneDrive\\ML_Models\\",
        f"C:\\Users\\{username}\\OneDrive\\Documents\\ML_Models\\",
        
        # Google Drive Desktop paths
        f"C:\\Users\\{username}\\Google Drive\\ML_Models\\",
        "G:\\My Drive\\ML_Models\\",
        
        # Dropbox alternative
        f"C:\\Users\\{username}\\Dropbox\\ML_Models\\",
        
        # Local fallback in your workspace
        ".\\model_backups\\",
    ]
    
    print("🔍 Searching for cloud storage paths...")
    
    for path in possible_paths:
        try:
            # Check if parent directory exists
            parent = os.path.dirname(path) if not path.endswith('\\') else path[:-1]
            
            if os.path.exists(parent) or path.startswith('.\\'):
                os.makedirs(path, exist_ok=True)
                path_type = "OneDrive" if "OneDrive" in path else ("Google Drive" if "Google Drive" in path else "Local")
                print(f"✅ Found {path_type}: {path}")
                return path
        except Exception as e:
            print(f"⚠️  Could not access {path}: {e}")
            continue
    
    # Final fallback
    fallback = ".\\model_backups\\"
    os.makedirs(fallback, exist_ok=True)
    print(f"⚠️  Using local fallback: {fallback}")
    return fallback

# Test the setup
recommended_path = setup_google_drive_path()
print(f"\n🎯 Recommended save path: {recommended_path}")
print("\n💡 Tips:")
print("  • For Google Colab: Enable 'Mount Drive' in the sidebar")
print("  • For local use: Install Google Drive Desktop app")
print("  • For OneDrive: The path should be automatically detected")
print("  • Manual setup: Edit GDRIVE_PATH variable in the save code")

# 🔧 Key Parameters for Experimenting with Different Configurations

## 1. LoRA Configuration Parameters

### **LoRA Rank (r)** - Most Important
- **Current**: `r=8` (conservative)
- **Options**: `4, 8, 16, 32, 64`
- **Impact**: Higher rank = more parameters = better adaptation but more memory
- **Recommendation**: Start with 8-16, increase if underfitting

### **LoRA Alpha** - Second Most Important  
- **Current**: `lora_alpha=16`
- **Formula**: Usually `lora_alpha = 2 * r` or `lora_alpha = r`
- **Impact**: Controls LoRA scaling, higher = stronger adaptation
- **Options**: `8, 16, 32, 64`

### **Target Modules** - Architecture Dependent
- **Current**: `["q", "v"]` (conservative)
- **Options for T5**: 
  - Minimal: `["q", "v"]`
  - Standard: `["q", "v", "k", "o"]` 
  - Comprehensive: `["q", "v", "k", "o", "wi_0", "wi_1", "wo"]`
- **Impact**: More modules = better adaptation but more parameters

### **LoRA Dropout**
- **Current**: `0.05`
- **Options**: `0.0, 0.05, 0.1, 0.2`
- **Impact**: Regularization, higher = less overfitting

---

## 2. Quantization Parameters

### **Compute Dtype** - Stability vs Speed
- **Current**: `torch.bfloat16` (most stable)
- **Options**: 
  - `torch.float16` (faster, less stable)
  - `torch.bfloat16` (balanced)
  - `torch.float32` (most stable, slower)

### **Double Quantization**
- **Current**: `False` (for stability)
- **Options**: `True/False`
- **Impact**: `True` saves memory but can cause instability

---

## 3. Training Arguments

### **Learning Rate** - Critical for Convergence
- **Current**: `1e-4`
- **Options**: `5e-5, 1e-4, 2e-4, 3e-4, 5e-4`
- **Rule**: Lower for stable training, higher for faster convergence

### **Batch Size & Gradient Accumulation**
- **Current**: `batch_size=2, grad_accum=8` (effective=16)
- **Options**:
  - Small GPU: `batch=1, accum=16`
  - Medium GPU: `batch=2, accum=8` 
  - Large GPU: `batch=4, accum=4`

### **Learning Rate Scheduler**
- **Current**: Default (linear decay)
- **Options**: `"linear", "cosine", "polynomial", "constant"`
- **Add**: `lr_scheduler_type="cosine"` to TrainingArguments

### **Warmup Steps**
- **Current**: Not set
- **Options**: `50, 100, 200` steps
- **Add**: `warmup_steps=100` to TrainingArguments

---

## 4. Data Processing Parameters

### **Max Length**
- **Current**: `512` tokens
- **Options**: `256, 384, 512, 768`
- **Impact**: Longer = more context but more memory

### **Preprocessing Prompt**
- **Current**: `"detoxify: {text}"`
- **Options**:
  - `"Rewrite to be non-toxic: {text}"`
  - `"Make this text neutral: {text}"`
  - `"Remove toxicity: {text}"`

---

## 5. Advanced Training Parameters

### **Gradient Clipping**
- **Current**: `max_grad_norm=1.0`
- **Options**: `0.5, 1.0, 2.0`

### **Weight Decay**
- **Current**: Default (0.01)
- **Options**: `0.0, 0.01, 0.1`
- **Add**: `weight_decay=0.1` to TrainingArguments

### **Adam Optimizer Parameters**
- **Add to TrainingArguments**:
  - `adam_beta1=0.9`
  - `adam_beta2=0.999`
  - `adam_epsilon=1e-8`

In [ ]:
# 🧪 CONFIGURATION TEMPLATES - Ready to Copy & Modify

# ========================================
# TEMPLATE 1: AGGRESSIVE CONFIGURATION
# ========================================
def create_aggressive_config():
    """Higher capacity, faster learning - for underfitting issues"""
    
    # Quantization config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,  # Re-enable for memory efficiency
    )
    
    # LoRA config - More parameters
    lora_config = LoraConfig(
        r=16,  # Increased rank
        lora_alpha=32,  # Increased alpha
        target_modules=["q", "v", "k", "o", "wi_0", "wi_1", "wo"],  # All modules
        lora_dropout=0.1,  # Higher dropout
        bias="none",
        task_type=TaskType.SEQ_2_SEQ_LM
    )
    
    # Training args - Faster learning
    training_args = TrainingArguments(
        output_dir="./mt5-detoxify-aggressive",
        per_device_train_batch_size=4,  # Larger batch if GPU allows
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,  # Higher learning rate
        bf16=True,
        logging_steps=5,
        save_steps=500,
        eval_strategy="no",
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        gradient_checkpointing=True,
        max_grad_norm=1.0,
        warmup_steps=100,  # Add warmup
        lr_scheduler_type="cosine",  # Cosine scheduler
        weight_decay=0.01,
    )
    
    return bnb_config, lora_config, training_args

# ========================================
# TEMPLATE 2: CONSERVATIVE CONFIGURATION  
# ========================================
def create_conservative_config():
    """Lower capacity, stable training - for overfitting or instability"""
    
    # Quantization config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False,  # Disabled for stability
    )
    
    # LoRA config - Fewer parameters
    lora_config = LoraConfig(
        r=4,  # Very low rank
        lora_alpha=8,  # Low alpha
        target_modules=["q", "v"],  # Minimal modules
        lora_dropout=0.05,  # Low dropout
        bias="none",
        task_type=TaskType.SEQ_2_SEQ_LM
    )
    
    # Training args - Slow and stable
    training_args = TrainingArguments(
        output_dir="./mt5-detoxify-conservative",
        per_device_train_batch_size=1,  # Small batch
        gradient_accumulation_steps=16,  # More accumulation
        num_train_epochs=5,  # More epochs
        learning_rate=5e-5,  # Lower learning rate
        bf16=True,
        logging_steps=10,
        save_steps=500,
        eval_strategy="no",
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        gradient_checkpointing=True,
        max_grad_norm=0.5,  # Stricter clipping
        warmup_steps=200,  # Longer warmup
        lr_scheduler_type="linear",
        weight_decay=0.1,  # Higher regularization
    )
    
    return bnb_config, lora_config, training_args

# ========================================
# TEMPLATE 3: BALANCED CONFIGURATION
# ========================================
def create_balanced_config():
    """Middle ground - good starting point"""
    
    # Quantization config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False,
    )
    
    # LoRA config - Moderate parameters
    lora_config = LoraConfig(
        r=12,  # Medium rank
        lora_alpha=24,  # Proportional alpha
        target_modules=["q", "v", "k", "o"],  # Core attention modules
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.SEQ_2_SEQ_LM
    )
    
    # Training args - Balanced
    training_args = TrainingArguments(
        output_dir="./mt5-detoxify-balanced",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        num_train_epochs=3,
        learning_rate=1.5e-4,  # Medium learning rate
        bf16=True,
        logging_steps=5,
        save_steps=500,
        eval_strategy="no",
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        gradient_checkpointing=True,
        max_grad_norm=1.0,
        warmup_steps=100,
        lr_scheduler_type="cosine",
        weight_decay=0.05,
    )
    
    return bnb_config, lora_config, training_args

print("✅ Configuration templates created!")
print("📋 Available templates:")
print("  - create_aggressive_config(): High capacity, fast learning")
print("  - create_conservative_config(): Low capacity, stable training") 
print("  - create_balanced_config(): Middle ground approach")
print("\n🔄 To use a template:")
print("  bnb_config, lora_config, training_args = create_balanced_config()")
print("  # Then recreate model and trainer with new configs")

In [ ]:
# 🚀 QUICK EXPERIMENT HELPER
def create_model_with_config(config_name="balanced"):
    """Quick way to create a new model with different configurations"""
    
    # Clear previous model
    import gc
    if 'model_lora_stable' in globals():
        del model_lora_stable
    if 'model_quantized_stable' in globals():
        del model_quantized_stable
    torch.cuda.empty_cache()
    gc.collect()
    
    # Get configuration
    if config_name == "aggressive":
        bnb_config, lora_config, training_args = create_aggressive_config()
    elif config_name == "conservative":
        bnb_config, lora_config, training_args = create_conservative_config()
    elif config_name == "balanced":
        bnb_config, lora_config, training_args = create_balanced_config()
    else:
        raise ValueError("config_name must be 'aggressive', 'conservative', or 'balanced'")
    
    print(f"🔧 Creating model with {config_name} configuration...")
    
    # Load model with new config
    model_quantized = T5ForConditionalGeneration.from_pretrained(
        "google/mt5-base",
        quantization_config=bnb_config,
        device_map="auto"
    )
    
    # Prepare for training
    model_quantized = prepare_model_for_kbit_training(model_quantized)
    
    # Apply LoRA
    model_lora = get_peft_model(model_quantized, lora_config)
    
    # Create data collator
    data_collator = DataCollatorForSeq2Seq(
        TOKENIZER, 
        model=model_lora,
        padding=True,
        return_tensors="pt"
    )
    
    # Create trainer
    trainer = Trainer(
        model=model_lora,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
        callbacks=[debug_callback]
    )
    
    # Test model stability
    print("🧪 Testing model stability...")
    test_batch = data_collator([tokenized_dataset[0]])
    model_device = next(model_lora.parameters()).device
    test_batch = {k: v.to(model_device) if isinstance(v, torch.Tensor) else v for k, v in test_batch.items()}
    
    model_lora.eval()
    with torch.no_grad():
        outputs = model_lora(
            input_ids=test_batch['input_ids'],
            attention_mask=test_batch['attention_mask'],
            labels=test_batch['labels']
        )
        
        loss_value = outputs.loss.item()
        has_nan = torch.isnan(outputs.logits).any().item()
        
        print(f"✅ Model created successfully!")
        print(f"📊 Test loss: {loss_value:.4f}")
        print(f"🔍 Contains NaN: {has_nan}")
        
        if has_nan or loss_value != loss_value:  # NaN check
            print("⚠️  Warning: Model shows instability")
        else:
            print("✅ Model is stable and ready for training")
    
    model_lora.train()
    
    # Print configuration summary
    trainable_params = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model_lora.parameters())
    
    print(f"\n📋 Configuration Summary ({config_name}):")
    print(f"  LoRA rank: {lora_config.r}")
    print(f"  LoRA alpha: {lora_config.lora_alpha}")
    print(f"  Target modules: {lora_config.target_modules}")
    print(f"  Learning rate: {training_args.learning_rate}")
    print(f"  Batch size: {training_args.per_device_train_batch_size}")
    print(f"  Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
    
    return model_lora, trainer

# Example usage:
print("🎯 Usage examples:")
print("  model, trainer = create_model_with_config('balanced')")
print("  model, trainer = create_model_with_config('aggressive')")  
print("  model, trainer = create_model_with_config('conservative')")
print("  trainer.train()  # Start training")

# 🔬 Float16 vs BFloat16: Complete Technical Comparison

## 📊 **Bit Representation & Precision**

### **Float16 (Half Precision)**
- **Total bits**: 16
- **Sign bit**: 1
- **Exponent**: 5 bits (bias = 15)
- **Mantissa**: 10 bits
- **Range**: ±65,504 (limited)
- **Precision**: ~3-4 decimal digits

### **BFloat16 (Brain Float)**
- **Total bits**: 16  
- **Sign bit**: 1
- **Exponent**: 8 bits (bias = 127) - **Same as Float32!**
- **Mantissa**: 7 bits
- **Range**: ±3.4×10³⁸ (same as Float32)
- **Precision**: ~2-3 decimal digits

---

## ⚖️ **Key Trade-offs**

| Aspect | Float16 | BFloat16 | Winner |
|--------|---------|----------|---------|
| **Numerical Range** | Limited (±65K) | Wide (±3.4×10³⁸) | 🏆 **BFloat16** |
| **Precision** | Higher (10-bit mantissa) | Lower (7-bit mantissa) | 🏆 **Float16** |
| **Overflow Risk** | High | Low | 🏆 **BFloat16** |
| **Underflow Risk** | High | Low | 🏆 **BFloat16** |
| **Hardware Support** | Limited | Growing (TPUs, modern GPUs) | 🏆 **BFloat16** |
| **Memory Usage** | Same (16 bits) | Same (16 bits) | 🤝 **Tie** |

---

## 🧠 **Why BFloat16 is Better for Deep Learning**

### **1. Dynamic Range**
```
Float16:  Can represent values from ±6×10⁻⁸ to ±65,504
BFloat16: Can represent values from ±1×10⁻³⁸ to ±3.4×10³⁸
```

### **2. Gradient Stability**
- **Float16**: Gradients often underflow to zero (vanishing gradients)
- **BFloat16**: Maintains gradient magnitudes better during backpropagation

### **3. Loss Landscape**
- **Float16**: Can cause loss spikes due to overflow
- **BFloat16**: More stable loss curves, fewer NaN issues

---

## 🔍 **In Your mT5 Training Context**

### **Why You Switched from Float16 to BFloat16:**

1. **NaN Loss Issue**: Your original `float16` config caused NaN logits
2. **Quantization Stability**: 4-bit quantization + float16 = numerical instability
3. **LoRA Adaptation**: Low-rank adaptations are sensitive to precision

# 🎯 **When to Use Each Format**

## 🟢 **Use BFloat16 When:**
- Training large language models (like your mT5)
- Using quantization (QLoRA, 4-bit, 8-bit)
- Working with transformers/attention mechanisms
- Need stable gradients
- Using modern GPUs (A100, H100, RTX 40xx series)
- Training is producing NaN losses with Float16

## 🟡 **Use Float16 When:**
- Training smaller models (CNNs, simple MLPs)
- Maximum precision is critical
- Working with older hardware
- Memory is extremely constrained
- Your training is already stable

## 🔴 **Avoid Float16 When:**
- Training very deep networks
- Using aggressive optimizations (quantization + low precision)
- Gradients are vanishing
- Getting overflow/underflow errors

---

# 📈 **Performance Impact in Your Case**

## **Memory Usage**: 
- Both use exactly **16 bits per parameter**
- No difference in memory consumption

## **Speed**:
- **Modern GPUs**: BFloat16 often faster (hardware optimized)
- **Older GPUs**: Float16 might be slightly faster
- **TPUs**: BFloat16 significantly faster (native support)

## **Training Stability**:
- **Your original issue**: Float16 + 4-bit quantization = NaN disaster
- **Current solution**: BFloat16 + 4-bit quantization = stable training ✅

---

# 🔧 **Configuration Impact**

```python
# ❌ PROBLEMATIC (what you had before)
bnb_config = BitsAndBytesConfig(
    bnb_4bit_compute_dtype=torch.float16,  # Caused NaN
    bnb_4bit_use_double_quant=True,        # Added instability
)
training_args = TrainingArguments(fp16=True)  # Amplified the problem

# ✅ STABLE (what you have now) 
bnb_config = BitsAndBytesConfig(
    bnb_4bit_compute_dtype=torch.bfloat16,  # Stable
    bnb_4bit_use_double_quant=False,        # Conservative
)
training_args = TrainingArguments(bf16=True)   # Consistent
```

---

# 🏆 **Bottom Line for Your mT5 Training**

**BFloat16 is the superior choice because:**
1. **Wider dynamic range** prevents overflow/underflow
2. **Better gradient preservation** during backpropagation  
3. **More stable with quantization** (your 4-bit setup)
4. **Industry standard** for large language model training
5. **Hardware optimized** on modern accelerators

Your switch from `float16` to `bfloat16` was the key fix that solved your NaN loss problem! 🎉